#**CHAPTER 4. GRAPH BASED MODELS**
---

##0.REFERENCE

https://claude.ai/share/f79e007e-9471-4e6b-a465-adc17a3a020c

##1.CONTEXT

**Chapter 4 — Graph-Based Models: Relational Inference**

**Where We Are in the Series**

The first three chapters built governance discipline layer by layer. Chapter 1 enforced the distinction between patterns and facts in unsupervised clustering through schema validation, stability testing, and abstention gates. Chapter 2 scaled those controls to match the increased capability of nonlinear supervised models, where optimization pressure introduced leakage, proxy exploitation, and temporal decay. Chapter 3 coupled two generative architectures into a single governed system, treating the latent space as an interface artifact requiring its own validation protocol.

Each chapter followed the same thesis: **Capability ↑ ⇒ Risk ↑ ⇒ Controls must ↑.** Each introduced a risk category the previous controls were not designed to catch, and each extended the artifact bundle and decision gate accordingly.

Chapter 4 introduces **relational risk.** When data is structured as a graph — entities connected by typed edges — the model does not learn from rows in isolation. It learns from neighborhoods, from paths, from the topology of connection itself. That shift from instance-level to relational learning changes what the model can detect, and what can go wrong.

**The Objective of This Notebook**

The notebook has three learning objectives, each designed to extend the governance competencies built in earlier chapters into a relational setting.

The first objective is to understand why relationships become signals. In tabular settings, each row stands alone. A customer's behavior is a function of that customer's features. In a graph, a customer's behavior is also a function of who that customer is connected to, how often, through which channels, and how that neighborhood itself is connected. Adjacency becomes a feature. This is the fundamental insight that makes graph models powerful — and the fundamental reason they require governance controls that tabular models do not. This notebook makes that conversion step visible and auditable. You will see exactly how an adjacency matrix becomes an embedding space (Capsule A) and how it becomes a message-passing substrate (Capsule B), and you will see the governance artifacts that document each transformation.

The second objective is to identify where relational risk emerges. Three specific risks are demonstrated, each tied to a structural property of the synthetic graph. **Propagation risk** arises because information — and bias — travels along edges. A GNN with two layers does not just look at a node's immediate neighbors; it aggregates signals from two hops away. If a biased signal enters the graph at any point within that radius, it propagates outward without any single edge being individually suspicious. **Boundary risk** arises because every graph is a subset of a larger possible graph. The edges that are present define what the model can see. The edges that are absent — whether because they were never observed, fell outside a time window, or were excluded by a governance rule — are invisible to the model but may carry information. The notebook tests what happens when the boundary shifts. **Proxy risk** arises because some edges carry ambiguous meaning. A "shared device" edge tells you two accounts accessed the same hardware. It does not tell you whether that access was coordinated, coincidental, or the result of a household sharing a single device. The notebook forces this ambiguity into the open by including a shared\_device edge type and then deliberately excluding it from the GNN adjacency while leaving it in the embedding input, demonstrating how the same edge can be legitimate in one context and dangerous in another.

The third objective is to see how governance artifacts constrain interpretation in a relational setting. The artifact bundle produced by this notebook is the largest in the series. It includes everything from previous chapters — schemas, validation logs, split manifests, metrics, stability reports, guardrails, decision gates, model cards, governance memos — plus artifacts that are specific to graph models: a graph construction manifest that documents which node and edge types are in scope and why, a propagation constraints file that records the maximum hop depth and the edge types permitted in message passing, and a sensitivity report that measures how embeddings and predictions shift under structural perturbation.

**The Examples Implemented**

The notebook builds a single synthetic typed graph and runs two governed capsules against it. The graph contains three node types — accounts, vendors, and devices — and three edge types — transaction (account to vendor), shared\_device (account to device), and co\_supply (vendor to vendor). Two nodes are deliberately constructed to demonstrate specific governance scenarios. A hub account is given an artificially high number of transaction edges, creating a spurious centrality signal. A connector account is given an artificially high number of shared\_device edges, creating a proxy dominance signal. Neither node is labeled as flagged. The point is not that they are suspicious. The point is that their structural position makes them visible to the model, and visibility is not the same as importance.

**Capsule A** implements graph embeddings through adjacency matrix factorization via truncated SVD. This is a deliberately simple method — no random walks, no iterative optimization beyond what scikit-learn provides internally. The simplicity is a feature, not a limitation. It isolates the governance questions from the modeling questions. You can understand exactly what the SVD is doing to the adjacency matrix, which means you can reason about exactly what the resulting embeddings represent and what they do not. The capsule then runs three stability tests: edge dropout removes a fraction of edges at random and measures whether the nearest-neighbor structure of the embeddings holds; edge noise adds spurious edges and measures the same; time-window shift excludes edges older than a cutoff and measures whether the embeddings change. Each perturbation produces a neighbor overlap score and a cosine drift score. If either metric breaches its threshold across any perturbation, interpretation is blocked for that capsule.

**Capsule B** implements a two-layer graph convolutional network manually in PyTorch, without any graph neural network library. The adjacency matrix is constructed from only the permitted edge types — transaction and co\_supply — and the shared\_device edges are excluded by governance rule, not by modeling convenience. The exclusion is documented in the propagation constraints artifact with an explicit justification. Node features include two synthetic continuous features and one intentionally proxy-like feature (a synthetic device count) that is flagged for human review. The GNN trains on a synthetic node classification task — predicting a review\_flag label that has been assigned to roughly eight percent of accounts at random. The label is teaching-only and is treated as unverified throughout. Sensitivity tests repeat the evaluation under edge dropout and time-window shift, measuring prediction flip rate and score drift. If either metric breaches its threshold, interpretation is blocked for the GNN.

**Expected Behavior and Results**

When you execute this notebook end to end, several things will happen in a predictable sequence. Validation will pass. The synthetic data generator is deterministic and schema-consistent by construction, so the fail-closed validation gate in Cell 5 will not trigger. If it does, something has gone wrong with the environment or the seed state, and the notebook will halt and write a blocked decision before raising an error.

Both capsules will produce embeddings and predictions. The embedding PCA plot will show three visible clusters corresponding to the three node types, because the adjacency structure separates them. The hub node and the connector node will be marked on that plot. Their positions will not necessarily be outliers — their structural anomaly is in degree, not necessarily in embedding location — but they will be visible for inspection. The GNN training loss will decrease over sixty epochs. The validation accuracy will fluctuate, particularly given the class imbalance (roughly eight percent positive labels), and should not be interpreted as evidence of model quality. The confusion matrix will likely show that the model predicts the majority class most of the time. This is expected. The synthetic label has no real signal; the model is fitting noise in the neighborhood structure.

The stability tests are where the governance logic becomes consequential. The decision gate in Cell 8 evaluates whether either capsule's sensitivity metrics breached their thresholds. If they did, the overall status is set to **"abstain,"** meaning interpretation is not permitted without human review. If both capsules passed, the status is **"pass\_exploratory"** — but the required\_human\_review flag remains unconditionally true, and every output carries **"Not verified."**

Cell 10 will produce a sample output that includes a deliberately prohibited query — "Who should we investigate?" — and the notebook's refusal to answer it. This refusal is not a failure. It is the expected behavior of a governed system encountering a request outside its permitted boundary, and it is recorded as an enforcement event in the guardrails report.

The artifact bundle will contain eighteen files, written regardless of whether the decision status is pass, abstain, or blocked. The governance memo separates what is known from what is assumed, what is open from what has been decided. The model card states that **structure is not causality** and that **connectivity is not explanation.** These are not decorative disclaimers. They define the interpretive boundary of everything this notebook produces.

Read the governance memo before you read the plots. That is the order this series has been designed for.

##2.LIBRARIES AND ENVIRONMENT

In [ ]:
# =============================================================================
# CELL 2 — Install + Imports + Global Config + Helpers
# =============================================================================

# ---------- installs (only what Colab may lack) ----------
import subprocess, sys
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "networkx"])
# torch, scikit-learn, matplotlib, numpy, pandas are pre-installed in Colab

# ---------- standard imports ----------
import os, json, hashlib, time, platform, random, copy
from datetime import datetime, timezone, timedelta
import numpy as np
import pandas as pd
import networkx as nx
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, confusion_matrix
import matplotlib
matplotlib.use("Agg")          # non-interactive backend (safe in Colab pipelines)
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from scipy import sparse as sp

# ---------- GLOBAL_CONFIG ----------
GLOBAL_CONFIG = {
    # ── reproducibility ──
    "seed": 42,

    # ── graph sizes ──
    "n_accounts": 800,
    "n_vendors": 250,
    "n_devices": 200,

    # ── edge generation params ──
    "edge_params": {
        "account_vendor_p": 0.008,
        "account_device_p": 0.006,
        "vendor_vendor_p": 0.004,
        "weight_mean": 5.0,
        "weight_std": 3.0,
        "hub_account_id": 5,
        "hub_extra_edges": 60,
        "ambiguous_connector_id": 12,
        "connector_extra_device_edges": 25
    },

    # ── time window ──
    "time_window_params": {
        "start_days_ago": 180,
        "end_days_ago": 0
    },

    # ── Capsule A: Embeddings ──
    "embedding_dim": 32,
    "svd_n_components": 32,
    "stability_edge_dropout_p": 0.12,
    "stability_noise_edges": 30,
    "stability_time_cutoff_days": 90,
    "anchor_nodes_k": 8,
    "neighbor_k": 10,

    # ── Capsule B: GNN ──
    "gnn_hidden_dim": 64,
    "gnn_epochs": 60,
    "gnn_lr": 0.01,
    "gnn_weight_decay": 5e-4,
    "gnn_max_hops": 2,
    "gnn_allowed_edge_types": ["transaction", "co_supply"],
    "gnn_influence_clip": 1.0,
    "review_flag_rate": 0.08,

    # ── Stability thresholds ──
    "neighbor_overlap_min": 0.45,
    "score_drift_max": 0.35,
    "sensitivity_fail_policy": "abstain"
}

# ---------- set ALL seeds deterministically ----------
def set_all_seeds(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_all_seeds(GLOBAL_CONFIG["seed"])

# ---------- helper functions ----------
def stable_json_dump(path: str, obj):
    """Write JSON with sorted keys, pretty-print, UTF-8."""
    d = os.path.dirname(path)
    if d:
        os.makedirs(d, exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        json.dump(obj, f, indent=2, sort_keys=True, default=str)

def sha256_dict(obj) -> str:
    canonical = json.dumps(obj, sort_keys=True, default=str).encode("utf-8")
    return hashlib.sha256(canonical).hexdigest()

def sha256_file(path: str) -> str:
    h = hashlib.sha256()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(8192), b""):
            h.update(chunk)
    return h.hexdigest()

def now_utc_iso() -> str:
    return datetime.now(timezone.utc).isoformat()

def safe_mkdirs(path: str):
    os.makedirs(path, exist_ok=True)

def summarize_graph_stats(G) -> dict:
    node_types = {}
    for n, d in G.nodes(data=True):
        t = d.get("node_type", "unknown")
        node_types[t] = node_types.get(t, 0) + 1
    edge_types = {}
    for u, v, d in G.edges(data=True):
        t = d.get("edge_type", "unknown")
        edge_types[t] = edge_types.get(t, 0) + 1
    return {
        "n_nodes": G.number_of_nodes(),
        "n_edges": G.number_of_edges(),
        "node_type_counts": node_types,
        "edge_type_counts": edge_types,
        "is_directed": G.is_directed(),
        "density": round(nx.density(G), 6)
    }

print("✅ Cell 2 complete — imports, config, and helpers loaded.")
print(f"   Seed = {GLOBAL_CONFIG['seed']} | Graph target: "
      f"{GLOBAL_CONFIG['n_accounts']} accounts, "
      f"{GLOBAL_CONFIG['n_vendors']} vendors, "
      f"{GLOBAL_CONFIG['n_devices']} devices")


✅ Cell 2 complete — imports, config, and helpers loaded.
   Seed = 42 | Graph target: 800 accounts, 250 vendors, 200 devices


##3.RUN INITIALIZATION, FOLDER LAYOUT AND RUN MANIFEST

###3.1.OVERVIEW

**Cell 3 — Run Initialization + Folder Layout + Run Manifest**

This cell is the first thing that executes after imports and configuration, and it does something that looks invisible but is foundational to every cell that follows: it builds the scaffolding that makes the entire run auditable. Nothing in this notebook produces output before the audit trail exists. That ordering is a governance decision, not a convenience.

The cell begins by constructing a deterministic run identifier. This is not a random string. It is a combination of the current timestamp and a cryptographic hash of the configuration dictionary from Cell 2. That means two runs with identical configurations will differ only in their timestamp. Two runs at the same second with different configurations will differ in their hash. The run_id is the single identifier that ties every artifact produced in this notebook back to the exact parameters and environment that produced it.

Next, the cell creates the full directory tree where artifacts will be written: schemas, validation, split, metrics, reports, model, decision, risk, memo, and outputs. These ten subdirectories are not optional. They are created whether or not the notebook succeeds. If the notebook halts in Cell 5 due to a validation failure, the directory structure still exists, and the decision file inside it will say "blocked." The artifact record persists even when the analysis does not.

The environment fingerprint captures the exact versions of every library in use: Python, NumPy, pandas, NetworkX, scikit-learn, PyTorch, SciPy, and Matplotlib. This is written to disk, not printed to the console. If someone re-runs this notebook six months later on a different machine, the fingerprint will differ, and a reviewer can compare the two to determine whether environment drift could explain any difference in results.

The PATHS registry is a single dictionary that maps every artifact name to its full file path. Every subsequent cell writes artifacts by referencing PATHS rather than constructing paths manually. This is a deliberate centralisation. If the directory structure ever changes, only this cell changes. All downstream cells remain untouched.

The cell closes by writing three placeholder files: run_manifest.json, decision.json, and risk_log.json. The decision starts as "pending." The risk log starts empty. Both will be overwritten by later cells as the notebook progresses. The point of writing them here is not their content. It is their existence. At every moment during execution, a valid decision file exists on disk. If the notebook crashes between Cell 5 and Cell 8, the decision file says "pending" rather than being absent. Absence would be ambiguous. "Pending" is a statement.


###3.2.CODE AND IMPLEMENTATION

In [ ]:
# =============================================================================
# CELL 3 — Run Initialization + Folder Layout + Run Manifest
# =============================================================================

# ---------- deterministic run_id (NO true randomness) ----------
_config_hash_seed = sha256_dict(GLOBAL_CONFIG)[:8]
_timestamp_tag   = datetime.now(timezone.utc).strftime("%Y%m%dT%H%M%S")
RUN_ID           = f"run_{_timestamp_tag}_{_config_hash_seed}"

# ---------- base artifact root ----------
ARTIFACT_ROOT = os.path.join(".", "artifacts", RUN_ID)

# ---------- create EXACT required directory layout ----------
_subdirs = ["schemas", "validation", "split", "metrics",
            "reports", "model", "decision", "risk", "memo", "outputs"]
for sub in _subdirs:
    safe_mkdirs(os.path.join(ARTIFACT_ROOT, sub))

# ---------- environment fingerprint ----------
def _pkg_version(name):
    try:
        import importlib.metadata
        return importlib.metadata.version(name)
    except Exception:
        return "unknown"

ENV_FINGERPRINT = {
    "python_version": sys.version,
    "platform": platform.platform(),
    "numpy":      _pkg_version("numpy"),
    "pandas":     _pkg_version("pandas"),
    "networkx":   _pkg_version("networkx"),
    "scikit-learn": _pkg_version("scikit-learn"),
    "torch":      _pkg_version("torch"),
    "scipy":      _pkg_version("scipy"),
    "matplotlib": _pkg_version("matplotlib")
}

# ---------- config hash (config + env subset) ----------
CONFIG_HASH = sha256_dict({"config": GLOBAL_CONFIG,
                           "env_python": ENV_FINGERPRINT["python_version"],
                           "env_platform": ENV_FINGERPRINT["platform"]})

# ---------- artifact path registry (used everywhere downstream) ----------
PATHS = {
    "root":                          ARTIFACT_ROOT,
    "run_manifest":                  os.path.join(ARTIFACT_ROOT, "run_manifest.json"),
    "node_schema":                   os.path.join(ARTIFACT_ROOT, "schemas", "node_schema.json"),
    "edge_schema":                   os.path.join(ARTIFACT_ROOT, "schemas", "edge_schema.json"),
    "graph_construction_manifest":   os.path.join(ARTIFACT_ROOT, "schemas", "graph_construction_manifest.json"),
    "data_validation_log":           os.path.join(ARTIFACT_ROOT, "validation", "data_validation_log.json"),
    "split_manifest":                os.path.join(ARTIFACT_ROOT, "split", "split_manifest.json"),
    "baseline_metrics":              os.path.join(ARTIFACT_ROOT, "metrics", "baseline_metrics.json"),
    "train_metrics":                 os.path.join(ARTIFACT_ROOT, "metrics", "train_metrics.json"),
    "eval_metrics":                  os.path.join(ARTIFACT_ROOT, "metrics", "eval_metrics.json"),
    "propagation_constraints":       os.path.join(ARTIFACT_ROOT, "reports", "propagation_constraints.json"),
    "sensitivity_report":            os.path.join(ARTIFACT_ROOT, "reports", "sensitivity_report.json"),
    "guardrails_report":             os.path.join(ARTIFACT_ROOT, "reports", "guardrails_report.json"),
    "model_card":                    os.path.join(ARTIFACT_ROOT, "model", "model_card.json"),
    "decision":                      os.path.join(ARTIFACT_ROOT, "decision", "decision.json"),
    "risk_log":                      os.path.join(ARTIFACT_ROOT, "risk", "risk_log.json"),
    "governance_memo":               os.path.join(ARTIFACT_ROOT, "memo", "governance_memo.json"),
    "sample_outputs":                os.path.join(ARTIFACT_ROOT, "outputs", "sample_outputs.json"),
    "bundle_zip":                    os.path.join(ARTIFACT_ROOT, "artifacts_bundle.zip")
}

# ---------- write run_manifest.json ----------
RUN_MANIFEST = {
    "run_id": RUN_ID,
    "timestamp": now_utc_iso(),
    "config": GLOBAL_CONFIG,
    "config_hash": CONFIG_HASH,
    "env_fingerprint": ENV_FINGERPRINT,
    "artifact_paths": {k: v for k, v in PATHS.items() if k != "root"}
}
stable_json_dump(PATHS["run_manifest"], RUN_MANIFEST)

# ---------- initialize decision.json placeholder ----------
DECISION_PLACEHOLDER = {
    "run_id": RUN_ID,
    "overall_status": "pending",
    "interpretation_allowed_embeddings": None,
    "interpretation_allowed_gnn": None,
    "required_human_review": True,
    "verification_status": "Not verified",
    "notes": "Placeholder — will be finalised in Cell 8."
}
stable_json_dump(PATHS["decision"], DECISION_PLACEHOLDER)

# ---------- initialize risk_log.json placeholder ----------
RISK_LOG = {
    "run_id": RUN_ID,
    "verification_status": "Not verified",
    "events": []   # capsules will append here
}
stable_json_dump(PATHS["risk_log"], RISK_LOG)

# ---------- in-memory mutable stores (capsules mutate these) ----------
RISK_LOG_EVENTS   = []          # list; capsules append dicts
SENSITIVITY_DATA  = {}          # key: "embeddings" / "gnn"
GUARDRAILS_EVENTS = []          # enforcement events
TRAIN_METRICS     = {}
EVAL_METRICS      = {}

print("✅ Cell 3 complete — run initialised.")
print(f"   RUN_ID        = {RUN_ID}")
print(f"   CONFIG_HASH   = {CONFIG_HASH[:16]}…")
print(f"   Artifact root = {ARTIFACT_ROOT}")

✅ Cell 3 complete — run initialised.
   RUN_ID        = run_20260201T142447_d6e413a4
   CONFIG_HASH   = 6e7c6b6060052e63…
   Artifact root = ./artifacts/run_20260201T142447_d6e413a4


##4.SYNTHETIC GRAPH GENERATOR, SCHEMAS AND CONSTRUCTION MANIFEST

###4.1.OVERVIEW

**Cell 4 — Synthetic Graph Generator + Schemas + Construction Manifest**

This cell builds the graph that the rest of the notebook governs. It is the longest cell in the notebook, and that length is justified because it is doing three things simultaneously: generating data, documenting what that data means, and recording the decisions made about what to include and what to exclude. In a real-world system, these three activities would happen at different times by different people. Here they are shown together so that their relationship is visible.

The node generation phase creates three populations. Accounts get two continuous synthetic features and one feature that is deliberately, explicitly designed to be proxy-like: synth_proxy_device_count, drawn from a Poisson distribution. This feature is not a mistake. It is a teaching instrument. It exists to demonstrate what happens when a feature that correlates with connectivity enters a model without domain review. Vendors and devices receive the same continuous features but their proxy count is set to zero, because in the synthetic scenario only accounts interact with devices.

The review_flag label is assigned randomly to approximately eight percent of accounts. Two accounts are then explicitly unflagged: the hub node and the connector node. This is a subtle but important decision. If either special node happened to be flagged, the model might learn to associate high degree with the label, confounding the spurious centrality demonstration. By removing the flag from both, the cell ensures that the governance risks it is designed to demonstrate remain structurally clean.

The edge generation phase produces three typed edge populations through independent Bernoulli sampling. Each account-vendor pair has a small probability of producing a transaction edge. Each account-device pair has a small probability of producing a shared\_device edge. Each vendor-vendor pair has a small probability of producing a co\_supply edge. After random generation completes, two nodes are surgically augmented: the hub receives extra transaction edges, and the connector receives extra shared\_device edges. These are not random. They are forced structural anomalies placed into the graph to make specific governance points visible.

The schema artifacts are where this cell earns its governance value. The node schema documents every field, its type, its allowed range, and its provenance. The edge schema goes further: it includes an edge\_semantics section that describes the meaning of each edge type, its direction, and critically, its ambiguity. The shared\_device entry states explicitly that shared device access does not imply shared intent. The graph construction manifest records the scope boundary, the time window, and the missingness policy: edges that are absent are uninformative, not evidence of disconnection. A human reviewer reading these three artifacts can reconstruct exactly what this graph contains and why.


###4.2.CODE AND IMPLEMENTATION

In [ ]:
# =============================================================================
# CELL 4 — Synthetic Graph Generator + Schemas + Construction Manifest
# =============================================================================
set_all_seeds(GLOBAL_CONFIG["seed"])   # re-seed for reproducibility

cfg  = GLOBAL_CONFIG
ep   = cfg["edge_params"]
tp   = cfg["time_window_params"]
now  = datetime.now(timezone.utc)

# ---------- 1. Build node table ----------
accounts = pd.DataFrame({
    "node_id":   [f"acct_{i:04d}" for i in range(cfg["n_accounts"])],
    "node_type": "account",
    "synth_feature_1": np.round(np.random.randn(cfg["n_accounts"]), 4),
    "synth_feature_2": np.round(np.random.rand(cfg["n_accounts"]) * 10, 4),
    # intentionally proxy-like feature (for GNN governance demo)
    "synth_proxy_device_count": np.random.poisson(2, cfg["n_accounts"])
})
vendors  = pd.DataFrame({
    "node_id":   [f"vend_{i:04d}" for i in range(cfg["n_vendors"])],
    "node_type": "vendor",
    "synth_feature_1": np.round(np.random.randn(cfg["n_vendors"]), 4),
    "synth_feature_2": np.round(np.random.rand(cfg["n_vendors"]) * 10, 4),
    "synth_proxy_device_count": 0
})
devices  = pd.DataFrame({
    "node_id":   [f"dev_{i:04d}"  for i in range(cfg["n_devices"])],
    "node_type": "device",
    "synth_feature_1": np.round(np.random.randn(cfg["n_devices"]), 4),
    "synth_feature_2": np.round(np.random.rand(cfg["n_devices"]) * 10, 4),
    "synth_proxy_device_count": 0
})
node_df = pd.concat([accounts, vendors, devices], ignore_index=True)

# ---------- 2. Assign synthetic review_flag (accounts only, sparse) ----------
n_flagged = max(1, int(cfg["n_accounts"] * cfg["review_flag_rate"]))
flagged_indices = np.random.choice(cfg["n_accounts"], size=n_flagged, replace=False)
node_df["review_flag"] = 0
node_df.loc[flagged_indices, "review_flag"] = 1
# make sure hub and connector are NOT automatically flagged (to avoid confounding)
hub_idx          = node_df.index[node_df["node_id"] == f"acct_{ep['hub_account_id']:04d}"].tolist()
connector_idx    = node_df.index[node_df["node_id"] == f"acct_{ep['ambiguous_connector_id']:04d}"].tolist()
for idx in hub_idx + connector_idx:
    node_df.loc[idx, "review_flag"] = 0

# ---------- 3. Build edge list ----------
acct_ids  = node_df.loc[node_df["node_type"] == "account", "node_id"].tolist()
vend_ids  = node_df.loc[node_df["node_type"] == "vendor",  "node_id"].tolist()
dev_ids   = node_df.loc[node_df["node_type"] == "device",  "node_id"].tolist()

def _rand_timestamp():
    days_back = np.random.randint(tp["end_days_ago"], tp["start_days_ago"])
    return (now - timedelta(days=int(days_back))).isoformat()

edges = []   # list of dicts

# (a) account → vendor  "transaction"
for a in acct_ids:
    for v in vend_ids:
        if np.random.rand() < ep["account_vendor_p"]:
            w = max(1, round(np.random.normal(ep["weight_mean"], ep["weight_std"]), 2))
            edges.append({"src": a, "dst": v, "edge_type": "transaction",
                          "weight": w, "timestamp": _rand_timestamp()})

# (b) account ↔ device  "shared_device"  (ambiguous proxy channel)
for a in acct_ids:
    for d in dev_ids:
        if np.random.rand() < ep["account_device_p"]:
            edges.append({"src": a, "dst": d, "edge_type": "shared_device",
                          "weight": 1.0, "timestamp": _rand_timestamp()})

# (c) vendor ↔ vendor  "co_supply"
for i, v1 in enumerate(vend_ids):
    for v2 in vend_ids[i+1:]:
        if np.random.rand() < ep["vendor_vendor_p"]:
            edges.append({"src": v1, "dst": v2, "edge_type": "co_supply",
                          "weight": 1.0, "timestamp": _rand_timestamp()})

# ---------- 4. Force hub node (spurious centrality) ----------
hub_id = f"acct_{ep['hub_account_id']:04d}"
hub_targets = np.random.choice(vend_ids, size=min(ep["hub_extra_edges"], len(vend_ids)), replace=False)
for v in hub_targets:
    edges.append({"src": hub_id, "dst": v, "edge_type": "transaction",
                  "weight": round(np.random.uniform(1, 3), 2), "timestamp": _rand_timestamp()})

# ---------- 5. Force ambiguous connector (proxy dominance) ----------
conn_id   = f"acct_{ep['ambiguous_connector_id']:04d}"
conn_devs = np.random.choice(dev_ids, size=min(ep["connector_extra_device_edges"], len(dev_ids)), replace=False)
for d in conn_devs:
    edges.append({"src": conn_id, "dst": d, "edge_type": "shared_device",
                  "weight": 1.0, "timestamp": _rand_timestamp()})

edge_df = pd.DataFrame(edges)
print(f"   Generated {len(node_df)} nodes, {len(edge_df)} edges")

# ---------- 6. Build NetworkX MultiDiGraph ----------
G = nx.MultiDiGraph()
for _, row in node_df.iterrows():
    G.add_node(row["node_id"],
               node_type=row["node_type"],
               synth_feature_1=float(row["synth_feature_1"]),
               synth_feature_2=float(row["synth_feature_2"]),
               synth_proxy_device_count=int(row["synth_proxy_device_count"]),
               review_flag=int(row["review_flag"]))
for _, row in edge_df.iterrows():
    G.add_edge(row["src"], row["dst"],
               edge_type=row["edge_type"],
               weight=float(row["weight"]),
               timestamp=row["timestamp"])

# ---------- 7. Write schemas ----------
NODE_SCHEMA = {
    "schema_version": "1.0",
    "source": "synthetic_generator",
    "rationale": "Typed node table for governance demonstration with account/vendor/device roles.",
    "fields": {
        "node_id":   {"type": "string", "pattern": "^(acct|vend|dev)_\\d{4}$", "required": True},
        "node_type": {"type": "string", "allowed": ["account", "vendor", "device"], "required": True},
        "synth_feature_1": {"type": "float", "required": True, "provenance": "synthetic_normal"},
        "synth_feature_2": {"type": "float", "min": 0.0, "max": 10.0, "required": True, "provenance": "synthetic_uniform"},
        "synth_proxy_device_count": {"type": "int", "min": 0, "required": True,
                                     "provenance": "synthetic_poisson",
                                     "governance_note": "Intentional proxy feature — demonstrates why feature provenance review is mandatory."},
        "review_flag": {"type": "int", "allowed": [0, 1], "required": True,
                        "provenance": "synthetic_assignment",
                        "governance_note": "Teaching label only. Not verified. Must not drive real decisions."}
    }
}
stable_json_dump(PATHS["node_schema"], NODE_SCHEMA)

EDGE_SCHEMA = {
    "schema_version": "1.0",
    "source": "synthetic_generator",
    "rationale": "Typed edge table preserving semantic distinctions for governance audit.",
    "fields": {
        "src":        {"type": "string", "required": True},
        "dst":        {"type": "string", "required": True},
        "edge_type":  {"type": "string", "allowed": ["transaction", "shared_device", "co_supply"], "required": True},
        "weight":     {"type": "float", "min": 0.0, "required": True},
        "timestamp":  {"type": "iso8601", "required": True}
    },
    "edge_semantics": {
        "transaction": {
            "direction": "account→vendor",
            "meaning": "Recorded commercial interaction (synthetic frequency).",
            "ambiguity_notes": "Weight is synthetic frequency; does not represent real monetary value."
        },
        "shared_device": {
            "direction": "account↔device (undirected semantically)",
            "meaning": "Two or more accounts accessed the same device (synthetic).",
            "ambiguity_notes": "HIGH AMBIGUITY — shared device does not imply shared intent, collusion, or fraud. This is the primary proxy-dominance risk channel. Excluded from GNN adjacency by default."
        },
        "co_supply": {
            "direction": "vendor↔vendor (undirected semantically)",
            "meaning": "Vendors share synthetic supply-chain attributes.",
            "ambiguity_notes": "Similarity edge; does not imply causal or contractual relationship."
        }
    }
}
stable_json_dump(PATHS["edge_schema"], EDGE_SCHEMA)

# ---------- 8. Write graph_construction_manifest ----------
GRAPH_CONSTRUCTION_MANIFEST = {
    "source": "synthetic_generator",
    "scope_boundary": {
        "included_node_types": ["account", "vendor", "device"],
        "included_edge_types": ["transaction", "shared_device", "co_supply"],
        "excluded_contexts": [
            "No real-world data included.",
            "No external APIs or databases queried.",
            "No privileged or PII data present."
        ]
    },
    "time_window": {
        "start": (now - timedelta(days=tp["start_days_ago"])).isoformat(),
        "end": now.isoformat(),
        "rule": "All edge timestamps fall within [start, end]. Time-window shift tests will move this boundary."
    },
    "missingness_semantics": {
        "policy": "Edges not present are treated as 'no observed interaction in window' — NOT as 'known absence'.",
        "implication": "Missing edges are uninformative, not evidence of disconnection."
    },
    "special_nodes": {
        "hub_node": {
            "id": hub_id,
            "role": "Forced high-degree account for spurious centrality demonstration.",
            "governance_note": "High degree does NOT imply importance or risk."
        },
        "ambiguous_connector": {
            "id": conn_id,
            "role": "Forced high shared_device degree for proxy dominance demonstration.",
            "governance_note": "Many shared devices does NOT imply shared intent."
        }
    },
    "graph_stats": summarize_graph_stats(G),
    "verification_status": "Not verified"
}
stable_json_dump(PATHS["graph_construction_manifest"], GRAPH_CONSTRUCTION_MANIFEST)

print("✅ Cell 4 complete — synthetic graph built and schemas written.")
print(f"   {summarize_graph_stats(G)}")

   Generated 1250 nodes, 2742 edges
✅ Cell 4 complete — synthetic graph built and schemas written.
   {'n_nodes': 1250, 'n_edges': 2742, 'node_type_counts': {'account': 800, 'vendor': 250, 'device': 200}, 'edge_type_counts': {'transaction': 1621, 'shared_device': 990, 'co_supply': 131}, 'is_directed': True, 'density': 0.001756}


##5.VALIDATION AND SPLIT MANIFEST. GRAPH VISUALIZATION

###5.1.OVERVIEW

**Cell 5 — Validation + Split Manifest + Graph Visualization**

This cell is the gatekeeper. Everything before it generates data and documents it. Everything after it models that data and interprets it. This cell sits between those two phases and asks a single question: is the data what it claims to be? If the answer is no, the notebook stops. Not with a warning. Not with a suggestion to proceed with caution. It stops completely, writes "blocked" to the decision file, and raises a SystemExit. This is fail-closed logic, and it is the first place in the notebook where the governance framework exercises real authority.

The validation runs twelve rules across three categories. Node validations check that all node types are in the allowed set, that all required fields are present, that node identifiers are unique, that synth\_feature\_2 falls within its declared range of zero to ten, that review\_flag contains only zero or one, and that the proxy feature is non-negative. Edge validations check that edge types are in the allowed set, that required fields are present, that all weights are positive, that every source and destination node actually exists in the node table, and that edge endpoints are type-consistent: transaction edges must connect accounts to vendors, shared\_device edges must connect accounts to devices, and co\_supply edges must connect vendors to vendors. Structural sanity checks flag any node whose degree exceeds five times the median (informational only, not blocking) and confirm that no self-loops exist.

Every rule is recorded in the validation log with its name, its pass/fail status, and a detail string. This log is written to disk before the fail-closed check executes. Even if the notebook halts, the log exists and a reviewer can read exactly which rule failed and why.

The split manifest creates two independent splits tailored to the two capsules. Capsule A needs an edge holdout for link-prediction evaluation, so fifteen percent of edges are set aside. Capsule B needs a node-level train/validation/test split among accounts only, so the account population is divided sixty-twenty-twenty. Each split is hashed. If someone re-runs the notebook with the same seed, the hashes will match. If a seed changes, the mismatch is detectable.

The visualization at the end of this cell draws an induced subgraph: forty randomly sampled accounts plus all of their one-hop neighbors. Nodes are colored by type. The plot is saved as a PNG and labeled "Not verified." Its purpose is not analysis. It is orientation. It shows the student what the graph looks like before any model touches it.


###5.2.CODE AND IMPLEMENTATION

In [ ]:

# =============================================================================
# CELL 5 — Validation + Split Manifest + Graph Visualization
# =============================================================================

validation_rules = []   # accumulate rule results

def _record(rule_name, passed, detail=""):
    validation_rules.append({
        "rule": rule_name,
        "passed": passed,
        "detail": detail
    })

# ---------- 1. Node validations ----------
# 1a. Type check: all node_types in allowed set
allowed_node_types = {"account", "vendor", "device"}
bad_ntypes = set(node_df["node_type"].unique()) - allowed_node_types
_record("node_type_allowed", len(bad_ntypes) == 0,
        f"Unexpected types: {bad_ntypes}" if bad_ntypes else "All node types valid.")

# 1b. Required fields present
required_node_fields = ["node_id", "node_type", "synth_feature_1", "synth_feature_2",
                        "synth_proxy_device_count", "review_flag"]
missing_nf = [f for f in required_node_fields if f not in node_df.columns]
_record("node_required_fields", len(missing_nf) == 0,
        f"Missing: {missing_nf}" if missing_nf else "All required node fields present.")

# 1c. node_id uniqueness
_record("node_id_unique", node_df["node_id"].is_unique,
        f"Duplicates: {node_df[node_df['node_id'].duplicated()]['node_id'].tolist()[:5]}"
        if not node_df["node_id"].is_unique else "All node_ids unique.")

# 1d. synth_feature_2 range [0, 10]
f2_ok = ((node_df["synth_feature_2"] >= 0.0) & (node_df["synth_feature_2"] <= 10.0)).all()
_record("node_feature_2_range", bool(f2_ok), "synth_feature_2 in [0,10].")

# 1e. review_flag only 0/1
rf_ok = node_df["review_flag"].isin([0, 1]).all()
_record("node_review_flag_values", bool(rf_ok), "review_flag ∈ {0,1}.")

# 1f. proxy feature non-negative
pf_ok = (node_df["synth_proxy_device_count"] >= 0).all()
_record("node_proxy_feature_nonneg", bool(pf_ok), "synth_proxy_device_count ≥ 0.")

# ---------- 2. Edge validations ----------
allowed_edge_types = {"transaction", "shared_device", "co_supply"}
bad_etypes = set(edge_df["edge_type"].unique()) - allowed_edge_types
_record("edge_type_allowed", len(bad_etypes) == 0,
        f"Unexpected edge types: {bad_etypes}" if bad_etypes else "All edge types valid.")

required_edge_fields = ["src", "dst", "edge_type", "weight", "timestamp"]
missing_ef = [f for f in required_edge_fields if f not in edge_df.columns]
_record("edge_required_fields", len(missing_ef) == 0,
        f"Missing: {missing_ef}" if missing_ef else "All required edge fields present.")

# 2a. Weight > 0
w_ok = (edge_df["weight"] > 0).all()
_record("edge_weight_positive", bool(w_ok), "All edge weights > 0.")

# 2b. Endpoints exist in node table
all_node_ids = set(node_df["node_id"])
bad_src = set(edge_df["src"]) - all_node_ids
bad_dst = set(edge_df["dst"]) - all_node_ids
_record("edge_endpoints_exist", len(bad_src) == 0 and len(bad_dst) == 0,
        f"Bad src: {list(bad_src)[:3]}, bad dst: {list(bad_dst)[:3]}"
        if (bad_src or bad_dst) else "All edge endpoints exist in node table.")

# 2c. Endpoint-type consistency
node_type_map = dict(zip(node_df["node_id"], node_df["node_type"]))
type_violations = []
for _, row in edge_df.iterrows():
    st, dt = node_type_map.get(row["src"]), node_type_map.get(row["dst"])
    if row["edge_type"] == "transaction" and not (st == "account" and dt == "vendor"):
        type_violations.append(f"{row['src']}→{row['dst']}")
    elif row["edge_type"] == "shared_device" and not (
            (st == "account" and dt == "device") or (st == "device" and dt == "account")):
        type_violations.append(f"{row['src']}→{row['dst']}")
    elif row["edge_type"] == "co_supply" and not (st == "vendor" and dt == "vendor"):
        type_violations.append(f"{row['src']}→{row['dst']}")
_record("edge_endpoint_type_consistency", len(type_violations) == 0,
        f"{len(type_violations)} violations (first 3: {type_violations[:3]})"
        if type_violations else "All endpoint-type pairs consistent.")

# ---------- 3. Structural sanity ----------
# 3a. Degree spike: flag if any node degree > 5× median
degrees = dict(G.degree())
deg_vals = np.array(list(degrees.values()))
median_deg = np.median(deg_vals)
spike_threshold = 5 * median_deg if median_deg > 0 else 50
spikes = [(n, d) for n, d in degrees.items() if d > spike_threshold]
_record("structural_degree_spike_check", True,   # informational, not blocking
        f"{len(spikes)} nodes exceed {spike_threshold:.0f} degree (median={median_deg:.1f}). "
        f"Examples: {spikes[:3]}. NOTE: hub node is by design.")

# 3b. Self-loop check (allowed in MultiDiGraph but flag if any)
self_loops = list(nx.selfloop_edges(G))
_record("structural_no_self_loops", len(self_loops) == 0,
        f"{len(self_loops)} self-loops found." if self_loops else "No self-loops.")

# ---------- 4. Write validation log ----------
all_passed = all(r["passed"] for r in validation_rules
                 # degree spike is informational — exclude from pass/fail
                 if r["rule"] != "structural_degree_spike_check")

VALIDATION_LOG = {
    "run_id": RUN_ID,
    "timestamp": now_utc_iso(),
    "overall_passed": all_passed,
    "rules": validation_rules,
    "verification_status": "Not verified"
}
stable_json_dump(PATHS["data_validation_log"], VALIDATION_LOG)

# ---------- 5. FAIL-CLOSED: if validation fails, block ----------
if not all_passed:
    DECISION_PLACEHOLDER["overall_status"] = "blocked"
    DECISION_PLACEHOLDER["notes"] = "Schema validation failed — notebook halted."
    stable_json_dump(PATHS["decision"], DECISION_PLACEHOLDER)
    raise SystemExit("🚨 VALIDATION FAILED — notebook blocked (fail-closed).")

print(f"   ✅ Validation passed ({len(validation_rules)} rules checked).")

# ---------- 6. Split manifest ----------
# For embeddings: edge holdout for link-pred proxy
set_all_seeds(GLOBAL_CONFIG["seed"])
edge_indices = np.arange(len(edge_df))
np.random.shuffle(edge_indices)
n_holdout = max(10, int(len(edge_df) * 0.15))
holdout_edge_idx  = edge_indices[:n_holdout].tolist()
train_edge_idx    = edge_indices[n_holdout:].tolist()

# For GNN: node train/val/test split (accounts with labels only)
acct_mask = node_df["node_type"] == "account"
acct_indices = node_df.index[acct_mask].tolist()
np.random.shuffle(acct_indices)
n_acct = len(acct_indices)
n_train = int(n_acct * 0.6)
n_val   = int(n_acct * 0.2)
gnn_train_idx = sorted(acct_indices[:n_train])
gnn_val_idx   = sorted(acct_indices[n_train:n_train+n_val])
gnn_test_idx  = sorted(acct_indices[n_train+n_val:])

SPLIT_MANIFEST = {
    "run_id": RUN_ID,
    "embeddings": {
        "method": "edge_holdout_link_pred_proxy",
        "train_edge_indices": train_edge_idx,
        "holdout_edge_indices": holdout_edge_idx,
        "train_hash": sha256_dict({"indices": train_edge_idx}),
        "holdout_hash": sha256_dict({"indices": holdout_edge_idx})
    },
    "gnn": {
        "method": "node_stratified_split_accounts_only",
        "train_indices": gnn_train_idx,
        "val_indices":   gnn_val_idx,
        "test_indices":  gnn_test_idx,
        "train_hash": sha256_dict({"indices": gnn_train_idx}),
        "val_hash":   sha256_dict({"indices": gnn_val_idx}),
        "test_hash":  sha256_dict({"indices": gnn_test_idx})
    },
    "verification_status": "Not verified"
}
stable_json_dump(PATHS["split_manifest"], SPLIT_MANIFEST)

# ---------- 7. VISUALIZATION: induced subgraph ----------
set_all_seeds(GLOBAL_CONFIG["seed"])
sample_accounts = np.random.choice(acct_ids, size=min(40, len(acct_ids)), replace=False).tolist()
# gather 1-hop neighbors
subgraph_nodes = set(sample_accounts)
for a in sample_accounts:
    for _, nbr, _ in G.out_edges(a, data=True):
        subgraph_nodes.add(nbr)
    for pred, _, _ in G.in_edges(a, data=True):
        subgraph_nodes.add(pred)
subG = G.subgraph(subgraph_nodes).copy()

# Convert to simple undirected for layout
simple_sub = nx.Graph()
for n, d in subG.nodes(data=True):
    simple_sub.add_node(n, **d)
for u, v, d in subG.edges(data=True):
    simple_sub.add_edge(u, v, **d)

color_map = {"account": "#4A90D9", "vendor": "#E67E22", "device": "#2ECC71"}
node_colors = [color_map.get(simple_sub.nodes[n].get("node_type", "unknown"), "#999")
               for n in simple_sub.nodes()]

fig, ax = plt.subplots(figsize=(11, 8))
pos = nx.spring_layout(simple_sub, seed=GLOBAL_CONFIG["seed"], k=2.2)
nx.draw_networkx_nodes(simple_sub, pos, node_color=node_colors, node_size=90, ax=ax, alpha=0.85)
nx.draw_networkx_edges(simple_sub, pos, ax=ax, alpha=0.25, width=0.7, edge_color="#666")

# legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=c, label=t) for t, c in color_map.items()]
ax.legend(handles=legend_elements, loc="upper left", fontsize=9, framealpha=0.9)

ax.set_title("Induced Subgraph — 40 Sampled Accounts + 1-Hop Neighbors\n"
             "(Synthetic data · Not verified)", fontsize=13, fontweight="bold")
ax.axis("off")
plt.tight_layout()
plt.savefig(os.path.join(ARTIFACT_ROOT, "graph_subgraph_viz.png"), dpi=130, bbox_inches="tight")
plt.show()
print("   📊 Graph visualization saved.")

# ---------- 8. Baseline metrics placeholder ----------
BASELINE_METRICS = {
    "run_id": RUN_ID,
    "graph_stats": summarize_graph_stats(G),
    "label_distribution": {
        "accounts_total": int((node_df["node_type"] == "account").sum()),
        "review_flag_1": int(node_df["review_flag"].sum()),
        "review_flag_0": int((node_df["review_flag"] == 0).sum())
    },
    "verification_status": "Not verified"
}
stable_json_dump(PATHS["baseline_metrics"], BASELINE_METRICS)

print("✅ Cell 5 complete — validation passed, splits created, subgraph drawn.")

   ✅ Validation passed (13 rules checked).
   📊 Graph visualization saved.
✅ Cell 5 complete — validation passed, splits created, subgraph drawn.


##6.GRAPH EMBEDDINGS

###6.1.OVERVIEW

**Cell 6 — Governed Capsule A: Graph Embeddings**

This cell executes the first complete governed model pipeline in the notebook. It takes the graph built in Cell 4, converts its structure into dense numerical vectors, evaluates those vectors, stress-tests them, checks whether they are stable enough to interpret, and records every step. The method is deliberately simple: truncated SVD applied directly to the adjacency matrix. No random walks, no iterative optimization beyond what scikit-learn provides internally. The simplicity is the point. When a method is simple enough to trace by hand, governance questions become answerable.

The adjacency matrix is built using only the training edges from the split manifest. The holdout edges are invisible to the embedding. SVD compresses the adjacency into a thirty-two-dimensional representation, and each row is L2-normalized so that cosine similarity becomes the natural distance metric. A link-prediction AUC is computed on the holdout edges as a secondary metric, but this is explicitly labeled secondary. It is a proxy check, not a performance claim.

The stability tests are where this cell demonstrates what governance actually does to an embedding pipeline. Three perturbations are applied independently. Edge dropout removes a random fraction of training edges and refits the embeddings. Edge noise adds spurious random edges and refits. Time-window shift excludes edges older than a cutoff and refits. For each perturbation, two metrics are computed: neighbor overlap at k, which measures whether the nearest neighbors of anchor nodes remain the same, and cosine drift, which measures how far each anchor's embedding vector has moved. If either metric breaches its threshold across any perturbation, interpretation is blocked for this capsule entirely.

The PCA visualization projects the thirty-two-dimensional embeddings into two dimensions. Three clusters appear, corresponding to the three node types. The hub node and connector node are marked with distinct symbols. Their positions are not necessarily outliers. Their structural anomaly lives in degree, not necessarily in embedding location. A student examining this plot must resist the temptation to read meaning into cluster membership. The clusters reflect adjacency structure, not behavior or intent. The stability bar charts show overlap and drift side by side across the three perturbations, with threshold lines drawn in red. A student reading these charts can see exactly whether this capsule passed or failed, and why. If a bar crosses the red line, interpretation is blocked. The chart does not suggest what to do next. It records what happened and leaves the next decision to the human reviewer who reads the governance memo.


###6.2.CODE AND IMPLEMENTATION

In [ ]:

# =============================================================================
# CELL 6 — Governed Capsule A: Graph Embeddings (End-to-End + Visuals)
# =============================================================================

def _build_adjacency(graph, node_list, edge_indices=None, edge_df_ref=None):
    """
    Build a symmetric adjacency matrix (scipy sparse) from the graph.
    If edge_indices + edge_df_ref are given, only those edges are used.
    Otherwise all edges in graph are used.
    """
    n = len(node_list)
    idx_map = {nid: i for i, nid in enumerate(node_list)}
    rows, cols, vals = [], [], []

    if edge_indices is not None and edge_df_ref is not None:
        for ei in edge_indices:
            row = edge_df_ref.iloc[ei]
            s, d = row["src"], row["dst"]
            if s in idx_map and d in idx_map:
                si, di = idx_map[s], idx_map[d]
                w = row["weight"]
                rows.extend([si, di]); cols.extend([di, si]); vals.extend([w, w])
    else:
        for u, v, d in graph.edges(data=True):
            if u in idx_map and v in idx_map:
                si, di = idx_map[u], idx_map[v]
                w = d.get("weight", 1.0)
                rows.extend([si, di]); cols.extend([di, si]); vals.extend([w, w])

    A = sp.csr_matrix((vals, (rows, cols)), shape=(n, n))
    return A

def _fit_embeddings(A, dim, seed):
    """SVD on adjacency → embedding matrix."""
    set_all_seeds(seed)
    svd = TruncatedSVD(n_components=dim, random_state=seed)
    emb = svd.fit_transform(A.toarray().astype(np.float64))
    # L2-normalise rows
    norms = np.linalg.norm(emb, axis=1, keepdims=True)
    norms[norms == 0] = 1.0
    emb = emb / norms
    return emb

def _top_k_neighbors(emb, anchor_idx, k):
    """Return indices of k nearest neighbours (cosine sim) excluding self."""
    sims = emb @ emb[anchor_idx]
    sims[anchor_idx] = -2.0   # exclude self
    return set(np.argsort(sims)[-k:].tolist())

# ---------- CAPSULE A MAIN ----------
set_all_seeds(GLOBAL_CONFIG["seed"])
cfg = GLOBAL_CONFIG
all_node_ids_ordered = list(G.nodes())   # stable order

# (1) Build adjacency with TRAIN edges only
A_train = _build_adjacency(G, all_node_ids_ordered,
                           edge_indices=SPLIT_MANIFEST["embeddings"]["train_edge_indices"],
                           edge_df_ref=edge_df)
emb_base = _fit_embeddings(A_train, cfg["svd_n_components"], cfg["seed"])
print(f"   Embeddings shape: {emb_base.shape}")

# (2) Link-prediction proxy on holdout edges (AUC — secondary metric)
holdout_edges = edge_df.iloc[SPLIT_MANIFEST["embeddings"]["holdout_edge_indices"]]
idx_map = {nid: i for i, nid in enumerate(all_node_ids_ordered)}

pos_scores, neg_scores = [], []
for _, row in holdout_edges.iterrows():
    si, di = idx_map.get(row["src"]), idx_map.get(row["dst"])
    if si is not None and di is not None:
        pos_scores.append(float(emb_base[si] @ emb_base[di]))
# negative samples: random non-edges
set_all_seeds(cfg["seed"])
for _ in range(len(pos_scores)):
    ri, rj = np.random.randint(0, len(all_node_ids_ordered), size=2)
    neg_scores.append(float(emb_base[ri] @ emb_base[rj]))

y_true  = [1]*len(pos_scores) + [0]*len(neg_scores)
y_score = pos_scores + neg_scores
try:
    link_pred_auc = round(float(roc_auc_score(y_true, y_score)), 4)
except Exception:
    link_pred_auc = None

# (3) Nearest-neighbour hypotheses for a few anchor nodes
set_all_seeds(cfg["seed"])
anchor_indices = np.random.choice(len(all_node_ids_ordered),
                                  size=min(cfg["anchor_nodes_k"], len(all_node_ids_ordered)),
                                  replace=False).tolist()
base_neighbors = {}   # anchor_idx → set of k-neighbor indices
for ai in anchor_indices:
    base_neighbors[ai] = _top_k_neighbors(emb_base, ai, cfg["neighbor_k"])

# ---------- (4) STABILITY / SENSITIVITY TESTS ----------
perturbation_results = []

def _perturb_and_eval(label, perturbed_A):
    """Refit embeddings on perturbed adjacency, compute overlap & drift."""
    emb_p = _fit_embeddings(perturbed_A, cfg["svd_n_components"], cfg["seed"])
    overlaps, drifts = [], []
    for ai in anchor_indices:
        nbrs_p = _top_k_neighbors(emb_p, ai, cfg["neighbor_k"])
        overlap = len(base_neighbors[ai] & nbrs_p) / cfg["neighbor_k"]
        drift   = float(1.0 - emb_base[ai] @ emb_p[ai])   # cosine distance
        overlaps.append(overlap)
        drifts.append(drift)
    mean_overlap = float(np.mean(overlaps))
    mean_drift   = float(np.mean(drifts))
    perturbation_results.append({
        "perturbation": label,
        "mean_neighbor_overlap_at_k": round(mean_overlap, 4),
        "mean_cosine_drift": round(mean_drift, 4),
        "per_anchor_overlaps": [round(x, 4) for x in overlaps],
        "per_anchor_drifts":   [round(x, 4) for x in drifts]
    })
    return mean_overlap, mean_drift

# (a) Edge dropout
set_all_seeds(cfg["seed"])
train_idx = SPLIT_MANIFEST["embeddings"]["train_edge_indices"]
keep_mask = np.random.rand(len(train_idx)) > cfg["stability_edge_dropout_p"]
dropped_idx = [idx for idx, keep in zip(train_idx, keep_mask) if keep]
A_dropout = _build_adjacency(G, all_node_ids_ordered,
                             edge_indices=dropped_idx, edge_df_ref=edge_df)
_perturb_and_eval("edge_dropout", A_dropout)

# (b) Edge noise: add random spurious edges
set_all_seeds(cfg["seed"])
noise_rows, noise_cols, noise_vals = [], [], []
n_nodes = len(all_node_ids_ordered)
for _ in range(cfg["stability_noise_edges"]):
    ri, rj = np.random.randint(0, n_nodes, size=2)
    noise_rows.extend([ri, rj]); noise_cols.extend([rj, ri]); noise_vals.extend([1.0, 1.0])
A_noise = A_train + sp.csr_matrix((noise_vals, (noise_rows, noise_cols)), shape=(n_nodes, n_nodes))
_perturb_and_eval("edge_noise", A_noise)

# (c) Time-window shift: exclude edges older than cutoff
cutoff_date = now - timedelta(days=cfg["stability_time_cutoff_days"])
time_filtered_idx = []
for ei in train_idx:
    row = edge_df.iloc[ei]
    try:
        ts = datetime.fromisoformat(row["timestamp"].replace("Z", "+00:00"))
        if ts >= cutoff_date:
            time_filtered_idx.append(ei)
    except Exception:
        time_filtered_idx.append(ei)   # keep if unparseable
A_time = _build_adjacency(G, all_node_ids_ordered,
                          edge_indices=time_filtered_idx, edge_df_ref=edge_df)
_perturb_and_eval("time_window_shift", A_time)

# ---------- (5) GUARDRAILS ----------
emb_interpretation_allowed = True
emb_guardrail_events = []

for pr in perturbation_results:
    if pr["mean_neighbor_overlap_at_k"] < cfg["neighbor_overlap_min"]:
        emb_interpretation_allowed = False
        emb_guardrail_events.append({
            "trigger": "neighbor_overlap_below_threshold",
            "perturbation": pr["perturbation"],
            "value": pr["mean_neighbor_overlap_at_k"],
            "threshold": cfg["neighbor_overlap_min"],
            "action": "interpretation_blocked_for_embeddings"
        })
    if pr["mean_cosine_drift"] > cfg["score_drift_max"]:
        emb_interpretation_allowed = False
        emb_guardrail_events.append({
            "trigger": "cosine_drift_above_threshold",
            "perturbation": pr["perturbation"],
            "value": pr["mean_cosine_drift"],
            "threshold": cfg["score_drift_max"],
            "action": "interpretation_blocked_for_embeddings"
        })

GUARDRAILS_EVENTS.extend(emb_guardrail_events)
SENSITIVITY_DATA["embeddings"] = perturbation_results

# ---------- (6) VISUALIZATIONS ----------
# --- 6a. 2D embedding projection (PCA) coloured by node_type ---
from sklearn.decomposition import PCA
pca = PCA(n_components=2, random_state=cfg["seed"])
emb_2d = pca.fit_transform(emb_base)

type_colors = {"account": "#4A90D9", "vendor": "#E67E22", "device": "#2ECC71"}
fig, ax = plt.subplots(figsize=(10, 7))
for ntype, color in type_colors.items():
    mask = np.array([G.nodes[nid].get("node_type") == ntype for nid in all_node_ids_ordered])
    ax.scatter(emb_2d[mask, 0], emb_2d[mask, 1],
               c=color, label=ntype, s=18, alpha=0.65, edgecolors="none")
# mark hub and connector
hub_i  = all_node_ids_ordered.index(f"acct_{cfg['edge_params']['hub_account_id']:04d}")
conn_i = all_node_ids_ordered.index(f"acct_{cfg['edge_params']['ambiguous_connector_id']:04d}")
ax.scatter(*emb_2d[hub_i],  marker="*", s=250, c="red",    zorder=5, edgecolors="k", label="Hub (spurious centrality)")
ax.scatter(*emb_2d[conn_i], marker="D", s=120, c="purple", zorder=5, edgecolors="k", label="Connector (proxy risk)")
ax.legend(fontsize=9, framealpha=0.9)
ax.set_title("Capsule A — Node Embeddings (PCA 2D)\n(Synthetic · Not verified)", fontsize=13, fontweight="bold")
ax.set_xlabel("PC 1"); ax.set_ylabel("PC 2")
plt.tight_layout()
plt.savefig(os.path.join(ARTIFACT_ROOT, "capsuleA_embedding_pca.png"), dpi=130, bbox_inches="tight")
plt.show()

# --- 6b. Stability plot: neighbor overlap@k across perturbations ---
perturb_labels = [pr["perturbation"] for pr in perturbation_results]
overlap_vals   = [pr["mean_neighbor_overlap_at_k"] for pr in perturbation_results]
drift_vals     = [pr["mean_cosine_drift"] for pr in perturbation_results]

fig, axes = plt.subplots(1, 2, figsize=(13, 4.5))
# overlap
axes[0].bar(perturb_labels, overlap_vals, color=["#5B9BD5","#ED7D31","#A5A5A5"], edgecolor="k", linewidth=0.8)
axes[0].axhline(cfg["neighbor_overlap_min"], color="red", linestyle="--", linewidth=1.5, label=f"Threshold ({cfg['neighbor_overlap_min']})")
axes[0].set_ylabel("Mean Neighbor Overlap @ k")
axes[0].set_title("Stability: Neighbor Overlap\n(Not verified)", fontsize=11, fontweight="bold")
axes[0].legend(fontsize=8)
axes[0].set_ylim(0, 1.05)
# drift
axes[1].bar(perturb_labels, drift_vals, color=["#5B9BD5","#ED7D31","#A5A5A5"], edgecolor="k", linewidth=0.8)
axes[1].axhline(cfg["score_drift_max"], color="red", linestyle="--", linewidth=1.5, label=f"Threshold ({cfg['score_drift_max']})")
axes[1].set_ylabel("Mean Cosine Drift")
axes[1].set_title("Stability: Cosine Drift\n(Not verified)", fontsize=11, fontweight="bold")
axes[1].legend(fontsize=8)
axes[1].set_ylim(0, max(drift_vals + [cfg["score_drift_max"]]) * 1.3)
plt.tight_layout()
plt.savefig(os.path.join(ARTIFACT_ROOT, "capsuleA_stability.png"), dpi=130, bbox_inches="tight")
plt.show()

# ---------- (7) UPDATE RISK LOG ----------
RISK_LOG_EVENTS.append({
    "capsule": "A_embeddings",
    "risk_category": "proxy_dominance",
    "detail": f"Node {cfg['edge_params']['ambiguous_connector_id']:04d} has high shared_device degree. "
              "shared_device edges are ambiguous proxies — interpretation must not treat connectivity as intent.",
    "severity": "warning"
})
RISK_LOG_EVENTS.append({
    "capsule": "A_embeddings",
    "risk_category": "spurious_centrality",
    "detail": f"Hub node acct_{cfg['edge_params']['hub_account_id']:04d} has artificially high degree. "
              "Embedding proximity to hub does NOT indicate shared risk.",
    "severity": "warning"
})
RISK_LOG_EVENTS.append({
    "capsule": "A_embeddings",
    "risk_category": "boundary_sensitivity",
    "detail": "Time-window shift perturbation was applied. If overlap/drift thresholds are breached, "
              "interpretation is blocked to prevent boundary-dependent conclusions.",
    "severity": "info"
})

# ---------- Save train metrics for embeddings ----------
TRAIN_METRICS["embeddings"] = {
    "method": "adjacency_svd",
    "n_components": cfg["svd_n_components"],
    "train_edges_used": len(SPLIT_MANIFEST["embeddings"]["train_edge_indices"]),
    "link_pred_auc_proxy": link_pred_auc,
    "interpretation_allowed": emb_interpretation_allowed,
    "verification_status": "Not verified"
}

print(f"✅ Cell 6 complete — Capsule A (Embeddings)")
print(f"   Link-pred AUC (proxy, secondary): {link_pred_auc}")
print(f"   Interpretation allowed: {emb_interpretation_allowed}")
print(f"   Guardrail events: {len(emb_guardrail_events)}")


   Embeddings shape: (1250, 32)
✅ Cell 6 complete — Capsule A (Embeddings)
   Link-pred AUC (proxy, secondary): 0.4514
   Interpretation allowed: False
   Guardrail events: 4


##7.GOVERNED GRAPH NEURAL NETWORK

###7.1.OVERVIEW

**Cell 7 — Governed Capsule B: Graph Neural Network**

This cell builds and trains a two-layer graph convolutional network from scratch in PyTorch, with no graph neural network library. Every operation — adjacency construction, normalization, message passing, gradient updates — is written explicitly. This is not an accident of tooling. It is a governance decision. When every step is visible in the code, every step can be audited. A model imported as a black-box function cannot be governed in the same way.

The adjacency matrix is filtered before it reaches the model. Only transaction and co\_supply edges are permitted. shared\_device edges are excluded by governance rule, and the exclusion is documented in a propagation constraints artifact with an explicit justification: shared\_device is an ambiguous proxy channel, and including it in message passing would propagate proxy signals without semantic grounding. This exclusion is the first place in the notebook where a governance rule directly shapes what the model can see. The influence of any single neighbor is also clipped to prevent one node from dominating the aggregation.

The node feature matrix includes three columns: two synthetic continuous features and the intentionally proxy-like synth_proxy_device_count. Its inclusion is not an oversight. It is a demonstration. The proxy feature is flagged in the guardrails as requiring human review. A student reading this cell learns that putting a proxy feature into a GNN is not a modeling error that a linter can catch. It is a governance question that only a domain expert can answer.

The GNN trains for sixty epochs on a synthetic binary classification task. The training loss decreases. The validation accuracy fluctuates. The test confusion matrix likely shows heavy prediction of the majority class. None of these results are surprising, because the label was assigned at random. The model is fitting noise in the neighborhood structure. The metrics are not evidence of capability. They are evidence that the pipeline executed correctly and that the governance workflow measured what it was designed to measure.

Sensitivity tests re-evaluate the trained model on perturbed adjacencies: one with edges dropped, one with the time window shifted. Score drift and prediction flip rate are measured. If either breaches its threshold, interpretation is blocked. The proxy feature warning is recorded as a guardrail event regardless of whether the sensitivity tests pass or fail. This ensures that the proxy risk is visible in the audit trail even when the model otherwise performs stably. A student reading this cell learns that stability and proxy risk are independent governance checks, and that passing one does not neutralize the other.


###7.2.CODE AND IMPLEMENTATION

In [6]:

# =============================================================================
# CELL 7 — Governed Capsule B: Graph Neural Network (End-to-End + Visuals)
# =============================================================================

# ---------- 1. Build FILTERED adjacency (only allowed edge types) ----------
allowed_etypes = set(cfg["gnn_allowed_edge_types"])   # excludes shared_device

def _build_gnn_adjacency(graph, node_list, allowed_types, edge_df_src=None, time_filter=None):
    """
    Build normalised adjacency for GCN.
    Only edges whose edge_type is in allowed_types are included.
    If time_filter (datetime) is given, only edges with timestamp >= time_filter.
    Clips influence to gnn_influence_clip after D^{-1/2} A D^{-1/2}.
    """
    n = len(node_list)
    idx_map = {nid: i for i, nid in enumerate(node_list)}
    rows, cols = [], []

    for u, v, d in graph.edges(data=True):
        if d.get("edge_type") not in allowed_types:
            continue
        if time_filter is not None:
            try:
                ts = datetime.fromisoformat(d.get("timestamp", "").replace("Z", "+00:00"))
                if ts < time_filter:
                    continue
            except Exception:
                pass
        if u in idx_map and v in idx_map:
            si, di = idx_map[u], idx_map[v]
            rows.extend([si, di]); cols.extend([di, si])   # symmetrise

    # add self-loops
    rows.extend(range(n)); cols.extend(range(n))
    vals = np.ones(len(rows))
    A = sp.csr_matrix((vals, (rows, cols)), shape=(n, n))

    # D^{-1/2} A D^{-1/2} normalisation
    deg = np.array(A.sum(axis=1)).flatten()
    deg_inv_sqrt = np.zeros_like(deg)
    nonzero = deg > 0
    deg_inv_sqrt[nonzero] = 1.0 / np.sqrt(deg[nonzero])
    D_inv_sqrt = sp.diags(deg_inv_sqrt)
    A_norm = D_inv_sqrt @ A @ D_inv_sqrt

    # influence clipping
    clip = cfg["gnn_influence_clip"]
    A_dense = A_norm.toarray()
    A_dense = np.clip(A_dense, 0, clip)
    return torch.tensor(A_dense, dtype=torch.float32)

set_all_seeds(cfg["seed"])
A_gnn_base = _build_gnn_adjacency(G, all_node_ids_ordered, allowed_etypes)

# ---------- 2. Node features ----------
# Use synth_feature_1, synth_feature_2, and intentionally the proxy feature
feature_cols = ["synth_feature_1", "synth_feature_2", "synth_proxy_device_count"]
# Build feature matrix aligned with all_node_ids_ordered
node_id_to_row = dict(zip(node_df["node_id"], node_df.index))
X_np = np.zeros((len(all_node_ids_ordered), len(feature_cols)), dtype=np.float32)
for i, nid in enumerate(all_node_ids_ordered):
    if nid in node_id_to_row:
        row_idx = node_id_to_row[nid]
        X_np[i] = node_df.loc[row_idx, feature_cols].values.astype(np.float32)
# z-score normalise
X_mean = X_np.mean(axis=0)
X_std  = X_np.std(axis=0)
X_std[X_std == 0] = 1.0
X_np = (X_np - X_mean) / X_std
X_tensor = torch.tensor(X_np, dtype=torch.float32)

# ---------- 3. Labels (accounts only) ----------
# Map node positions to labels; non-account nodes get label -1 (masked)
y_np = np.full(len(all_node_ids_ordered), -1, dtype=np.int64)
for i, nid in enumerate(all_node_ids_ordered):
    if nid in node_id_to_row:
        row_idx = node_id_to_row[nid]
        if node_df.loc[row_idx, "node_type"] == "account":
            y_np[i] = int(node_df.loc[row_idx, "review_flag"])
y_tensor = torch.tensor(y_np, dtype=torch.long)

# Train/val/test masks (node-level, based on all_node_ids_ordered positions)
nid_to_pos = {nid: i for i, nid in enumerate(all_node_ids_ordered)}
train_mask = torch.zeros(len(all_node_ids_ordered), dtype=torch.bool)
val_mask   = torch.zeros(len(all_node_ids_ordered), dtype=torch.bool)
test_mask  = torch.zeros(len(all_node_ids_ordered), dtype=torch.bool)

# gnn_train_idx etc. are DataFrame indices; map to graph positions
for df_idx in SPLIT_MANIFEST["gnn"]["train_indices"]:
    nid = node_df.loc[df_idx, "node_id"]
    if nid in nid_to_pos:
        train_mask[nid_to_pos[nid]] = True
for df_idx in SPLIT_MANIFEST["gnn"]["val_indices"]:
    nid = node_df.loc[df_idx, "node_id"]
    if nid in nid_to_pos:
        val_mask[nid_to_pos[nid]] = True
for df_idx in SPLIT_MANIFEST["gnn"]["test_indices"]:
    nid = node_df.loc[df_idx, "node_id"]
    if nid in nid_to_pos:
        test_mask[nid_to_pos[nid]] = True

# ---------- 4. Minimal GCN model ----------
class MinimalGCN(nn.Module):
    """2-layer GCN: input → hidden (ReLU) → output (2 classes)."""
    def __init__(self, in_dim, hidden_dim, out_dim):
        super().__init__()
        self.W1 = nn.Parameter(torch.randn(in_dim, hidden_dim) * 0.1)
        self.W2 = nn.Parameter(torch.randn(hidden_dim, out_dim) * 0.1)
        self.bias1 = nn.Parameter(torch.zeros(hidden_dim))
        self.bias2 = nn.Parameter(torch.zeros(out_dim))

    def forward(self, X, A):
        # Layer 1: A·X·W1 + b1, ReLU
        H = torch.relu(A @ X @ self.W1 + self.bias1)
        # Layer 2: A·H·W2 + b2  (no activation — logits)
        out = A @ H @ self.W2 + self.bias2
        return out

set_all_seeds(cfg["seed"])
model = MinimalGCN(in_dim=len(feature_cols),
                   hidden_dim=cfg["gnn_hidden_dim"],
                   out_dim=2)
optimizer = optim.Adam(model.parameters(), lr=cfg["gnn_lr"],
                       weight_decay=cfg["gnn_weight_decay"])
criterion = nn.CrossEntropyLoss()

# ---------- 5. Training loop ----------
train_losses, val_accs = [], []

for epoch in range(cfg["gnn_epochs"]):
    model.train()
    optimizer.zero_grad()
    logits = model(X_tensor, A_gnn_base)
    loss = criterion(logits[train_mask], y_tensor[train_mask])
    loss.backward()
    optimizer.step()
    train_losses.append(round(float(loss.item()), 6))

    # validation accuracy
    model.eval()
    with torch.no_grad():
        val_logits = model(X_tensor, A_gnn_base)
        val_preds  = val_logits[val_mask].argmax(dim=1)
        val_acc    = float((val_preds == y_tensor[val_mask]).float().mean().item())
    val_accs.append(round(val_acc, 4))

# ---------- 6. Evaluation on test set ----------
model.eval()
with torch.no_grad():
    test_logits = model(X_tensor, A_gnn_base)
    test_preds  = test_logits[test_mask].argmax(dim=1).numpy()
    test_true   = y_tensor[test_mask].numpy()

test_acc = round(float(accuracy_score(test_true, test_preds)), 4)
test_f1  = round(float(f1_score(test_true, test_preds, zero_division=0)), 4)
cm = confusion_matrix(test_true, test_preds)

# ---------- 7. Write propagation_constraints.json ----------
PROPAGATION_CONSTRAINTS = {
    "run_id": RUN_ID,
    "max_hops": cfg["gnn_max_hops"],
    "gcn_layers": cfg["gnn_max_hops"],
    "allowed_edge_types": list(allowed_etypes),
    "excluded_edge_types": ["shared_device"],
    "exclusion_justification": "shared_device is an ambiguous proxy channel. Including it in message-passing "
                                "would propagate proxy signals without semantic grounding.",
    "influence_clip": cfg["gnn_influence_clip"],
    "normalisation": "symmetric D^{-1/2} A D^{-1/2}",
    "self_loops": "included (standard GCN practice)",
    "verification_status": "Not verified"
}
stable_json_dump(PATHS["propagation_constraints"], PROPAGATION_CONSTRAINTS)

# ---------- 8. SENSITIVITY TESTS ----------
gnn_perturbation_results = []

def _eval_gnn_on_adj(A_perturbed, label):
    """Evaluate trained model on a perturbed adjacency. Measure score drift + flip rate."""
    model.eval()
    with torch.no_grad():
        base_logits = model(X_tensor, A_gnn_base)
        pert_logits = model(X_tensor, A_perturbed)
        base_preds  = base_logits[test_mask].argmax(dim=1).numpy()
        pert_preds  = pert_logits[test_mask].argmax(dim=1).numpy()
        # score drift: mean absolute difference of softmax probs
        base_probs = torch.softmax(base_logits[test_mask], dim=1).numpy()
        pert_probs = torch.softmax(pert_logits[test_mask], dim=1).numpy()
        score_drift = float(np.abs(base_probs - pert_probs).mean())
        flip_rate   = float((base_preds != pert_preds).mean())
    gnn_perturbation_results.append({
        "perturbation": label,
        "mean_score_drift": round(score_drift, 4),
        "prediction_flip_rate": round(flip_rate, 4)
    })
    return score_drift, flip_rate

# (a) Edge dropout — rebuild adjacency with dropout
set_all_seeds(cfg["seed"])
# We rebuild from graph but randomly drop edges
G_dropped = G.copy()
edges_to_drop = []
for u, v, k, d in list(G_dropped.edges(keys=True, data=True)):
    if d.get("edge_type") in allowed_etypes and np.random.rand() < cfg["stability_edge_dropout_p"]:
        edges_to_drop.append((u, v, k))
for u, v, k in edges_to_drop:
    G_dropped.remove_edge(u, v, k)
A_gnn_dropout = _build_gnn_adjacency(G_dropped, all_node_ids_ordered, allowed_etypes)
_eval_gnn_on_adj(A_gnn_dropout, "edge_dropout")

# (b) Time-window shift
time_cutoff = now - timedelta(days=cfg["stability_time_cutoff_days"])
A_gnn_time = _build_gnn_adjacency(G, all_node_ids_ordered, allowed_etypes, time_filter=time_cutoff)
_eval_gnn_on_adj(A_gnn_time, "time_window_shift")

SENSITIVITY_DATA["gnn"] = gnn_perturbation_results

# ---------- 9. GUARDRAILS ----------
gnn_interpretation_allowed = True
gnn_guardrail_events = []

for pr in gnn_perturbation_results:
    if pr["mean_score_drift"] > cfg["score_drift_max"]:
        gnn_interpretation_allowed = False
        gnn_guardrail_events.append({
            "trigger": "score_drift_above_threshold",
            "perturbation": pr["perturbation"],
            "value": pr["mean_score_drift"],
            "threshold": cfg["score_drift_max"],
            "action": "interpretation_blocked_for_gnn"
        })
    if pr["prediction_flip_rate"] > 0.20:   # hard-coded flip-rate guardrail
        gnn_interpretation_allowed = False
        gnn_guardrail_events.append({
            "trigger": "prediction_flip_rate_high",
            "perturbation": pr["perturbation"],
            "value": pr["prediction_flip_rate"],
            "threshold": 0.20,
            "action": "interpretation_blocked_for_gnn"
        })

# Proxy feature warning: synth_proxy_device_count is intentionally proxy-like
gnn_guardrail_events.append({
    "trigger": "proxy_feature_warning",
    "feature": "synth_proxy_device_count",
    "detail": "This feature is a proxy for device connectivity. Using it in GNN without "
              "domain validation risks proxy dominance. Flagged for human review.",
    "severity": "warning",
    "action": "flagged_for_review"
})

GUARDRAILS_EVENTS.extend(gnn_guardrail_events)

# ---------- 10. VISUALIZATIONS ----------
fig, axes = plt.subplots(1, 3, figsize=(17, 4.5))

# --- 10a. Training loss + validation accuracy ---
ax = axes[0]
ax.plot(train_losses, color="#4A90D9", linewidth=1.8, label="Train Loss")
ax.set_ylabel("Loss", color="#4A90D9"); ax.set_xlabel("Epoch")
ax.set_title("GNN Training Loss\n(Not verified)", fontsize=11, fontweight="bold")
ax2 = ax.twinx()
ax2.plot(val_accs, color="#E67E22", linewidth=1.5, linestyle="--", label="Val Accuracy")
ax2.set_ylabel("Val Accuracy", color="#E67E22")
ax.legend(loc="upper left", fontsize=8)
ax2.legend(loc="lower left", fontsize=8)

# --- 10b. Confusion matrix ---
ax = axes[1]
im = ax.imshow(cm, cmap="Blues", aspect="auto")
ax.set_xticks([0, 1]); ax.set_xticklabels(["Pred 0", "Pred 1"])
ax.set_yticks([0, 1]); ax.set_yticklabels(["True 0", "True 1"])
for i in range(2):
    for j in range(2):
        ax.text(j, i, str(cm[i, j]), ha="center", va="center",
                color="white" if cm[i, j] > cm.max()/2 else "black", fontsize=14)
ax.set_title("Confusion Matrix (Test)\n(Synthetic · Not verified)", fontsize=11, fontweight="bold")
fig.colorbar(im, ax=ax, shrink=0.8)

# --- 10c. Sensitivity: prediction flip rate ---
ax = axes[2]
perturb_names = [pr["perturbation"] for pr in gnn_perturbation_results]
flip_rates    = [pr["prediction_flip_rate"] for pr in gnn_perturbation_results]
drift_vals_g  = [pr["mean_score_drift"] for pr in gnn_perturbation_results]
x_pos = np.arange(len(perturb_names))
width = 0.35
bars1 = ax.bar(x_pos - width/2, flip_rates, width, color="#E74C3C", edgecolor="k", linewidth=0.8, label="Flip Rate")
bars2 = ax.bar(x_pos + width/2, drift_vals_g, width, color="#9B59B6", edgecolor="k", linewidth=0.8, label="Score Drift")
ax.axhline(0.20, color="red", linestyle="--", linewidth=1, label="Flip threshold (0.20)")
ax.axhline(cfg["score_drift_max"], color="purple", linestyle="--", linewidth=1, label=f"Drift threshold ({cfg['score_drift_max']})")
ax.set_xticks(x_pos); ax.set_xticklabels(perturb_names, fontsize=9)
ax.set_ylabel("Rate")
ax.set_title("GNN Sensitivity\n(Not verified)", fontsize=11, fontweight="bold")
ax.legend(fontsize=7, loc="upper right")
ax.set_ylim(0, max(flip_rates + drift_vals_g + [0.25, cfg["score_drift_max"]]) * 1.35)

plt.tight_layout()
plt.savefig(os.path.join(ARTIFACT_ROOT, "capsuleB_gnn_visuals.png"), dpi=130, bbox_inches="tight")
plt.show()

# ---------- 11. UPDATE RISK LOG ----------
RISK_LOG_EVENTS.append({
    "capsule": "B_gnn",
    "risk_category": "contagion_propagation",
    "detail": f"GCN uses {cfg['gnn_max_hops']}-hop message passing. Information (and bias) propagates "
              "through allowed edge types. Structural amplification is possible.",
    "severity": "warning"
})
RISK_LOG_EVENTS.append({
    "capsule": "B_gnn",
    "risk_category": "boundary_errors",
    "detail": "Time-window shift test changes which edges are in-scope. If predictions shift significantly, "
              "the model's output is boundary-dependent and unreliable for fixed decisions.",
    "severity": "warning"
})
RISK_LOG_EVENTS.append({
    "capsule": "B_gnn",
    "risk_category": "accountability_diffusion",
    "detail": "GNN scores are functions of neighbourhood structure. Attributing a score to a single node "
              "obscures the collective origin of the signal. Human review must trace provenance.",
    "severity": "warning"
})
RISK_LOG_EVENTS.append({
    "capsule": "B_gnn",
    "risk_category": "proxy_dominance",
    "detail": "synth_proxy_device_count is included as a node feature. This is a proxy for device connectivity "
              "and may dominate gradient flow. Requires domain sign-off before any real use.",
    "severity": "warning"
})

# ---------- Save metrics ----------
TRAIN_METRICS["gnn"] = {
    "method": "manual_gcn_pytorch",
    "layers": cfg["gnn_max_hops"],
    "hidden_dim": cfg["gnn_hidden_dim"],
    "epochs": cfg["gnn_epochs"],
    "final_train_loss": train_losses[-1],
    "train_losses_per_epoch": train_losses,
    "val_accs_per_epoch": val_accs,
    "verification_status": "Not verified"
}
EVAL_METRICS["gnn"] = {
    "test_accuracy": test_acc,
    "test_f1": test_f1,
    "confusion_matrix": cm.tolist(),
    "interpretation_allowed": gnn_interpretation_allowed,
    "verification_status": "Not verified"
}

print(f"✅ Cell 7 complete — Capsule B (GNN)")
print(f"   Test accuracy: {test_acc} | Test F1: {test_f1}")
print(f"   Interpretation allowed: {gnn_interpretation_allowed}")
print(f"   Guardrail events: {len(gnn_guardrail_events)}")



✅ Cell 7 complete — Capsule B (GNN)
   Test accuracy: 0.9375 | Test F1: 0.0
   Interpretation allowed: True
   Guardrail events: 1


##8.GUARDRAILS REPORT AND DETERMINISTIC DECISION LOGIC

###8.1.OVERVIEW

**Cell 8 — Guardrails Report + Deterministic Decision Logic**

This cell is where the notebook makes its decision. Not a recommendation. Not a suggestion. A deterministic, auditable verdict on whether the outputs of Capsules A and B are permitted to be interpreted. The logic is three rules, evaluated in order. If schema validation failed, the status is "blocked" — but if execution reached this cell, validation passed in Cell 5, so this rule is already satisfied. If either capsule failed its stability tests, or if a critical proxy warning was raised, the status is "abstain." Otherwise, the status is "pass\_exploratory." In every case, required\_human\_review is set to True. There is no path through this cell that produces a decision without mandatory human oversight. The decision artifact records not only the final status but the exact rule that produced it, so a reviewer can read the logic without re-running the notebook.

The guardrails report consolidates everything that was enforced during the run. It lists allowed uses — exploratory hypothesis generation, human-review prioritization, teaching — and prohibited uses — ranking entities for action, determining eligibility, adverse action, causal claims, normative labeling. It records every enforcement event from both capsules, including stability failures, proxy warnings, and the refusal example that will be added in Cell 10. It states two interpretation rules that govern everything this notebook has produced: structure is not causality, and connectivity is not explanation.

The sensitivity report collects the raw perturbation data from both capsules into a single artifact. The thresholds used to evaluate that data are recorded alongside it. A reviewer reading this report can see not only whether each perturbation passed or failed, but the exact values that produced that verdict and the exact thresholds against which they were compared.

The train and eval metrics files are written here rather than in the capsule cells because this cell is the consolidation point. It assembles metrics from both capsules into unified files, each stamped "Not verified." The risk log is finalized here as well, collecting all risk events from both capsules under five categories: relational bias and structural amplification, contagion, boundary errors, spurious centrality and proxy dominance, and accountability diffusion. These categories are not generic labels. Each one corresponds to a specific failure mode demonstrated somewhere in the notebook. A human reviewer reading the risk log can trace each event back to the capsule and the code section that produced it, which is the minimum condition for accountability in a governed system.


###8.2.CODE AND IMPLEMENTATION

In [8]:


# ---------- 1. Guardrails Report ----------
GUARDRAILS_REPORT = {
    "run_id": RUN_ID,
    "timestamp": now_utc_iso(),
    "allowed_use": [
        "Exploratory hypothesis generation only.",
        "Human-review prioritisation (e.g., selecting which hypotheses a domain expert examines first).",
        "Teaching and governance demonstration."
    ],
    "prohibited_use": [
        "Ranking entities for action or investigation.",
        "Determining eligibility, access, or creditworthiness.",
        "Adverse action against any individual or entity.",
        "Automated escalation without human review.",
        "Causal claims derived from graph structure.",
        "Normative labelling (e.g., 'suspicious', 'high-risk') without domain validation."
    ],
    "enforcement_events": GUARDRAILS_EVENTS,
    "interpretation_rules": {
        "fact_vs_hypothesis": "Any pattern observed in the graph (cluster, centrality, similarity) is a HYPOTHESIS. "
                              "It is NOT a fact until independently verified by domain experts.",
        "not_verified_label": "All interpretive outputs carry the label 'Not verified'.",
        "stability_prerequisite": "Interpretation is only permitted when stability tests pass for the relevant capsule.",
        "structure_is_not_causality": "Structure is not causality.",
        "connectivity_is_not_explanation": "Connectivity is not explanation."
    },
    "capsule_status": {
        "embeddings": {
            "interpretation_allowed": emb_interpretation_allowed,
            "guardrail_events_count": len([e for e in GUARDRAILS_EVENTS if "embeddings" in str(e).lower() or e.get("trigger","").startswith("neighbor") or e.get("trigger","").startswith("cosine")])
        },
        "gnn": {
            "interpretation_allowed": gnn_interpretation_allowed,
            "guardrail_events_count": len([e for e in GUARDRAILS_EVENTS if "gnn" in str(e).lower() or e.get("trigger","").startswith("score_drift") or e.get("trigger","").startswith("prediction") or e.get("trigger","").startswith("proxy")])
        }
    },
    "verification_status": "Not verified"
}
stable_json_dump(PATHS["guardrails_report"], GUARDRAILS_REPORT)

# ---------- 2. Deterministic Decision Logic ----------
# Rules (evaluated in order):
#   1. If schema validation failed → "blocked"  (handled in Cell 5; if we're here, it passed)
#   2. If EITHER capsule fails stability OR proxy_dominance is flagged critical → "abstain"
#   3. If BOTH capsules pass stability and no critical warnings → "pass_exploratory"

has_critical_proxy = any(
    e.get("trigger") == "proxy_feature_warning" and e.get("severity") == "warning"
    for e in GUARDRAILS_EVENTS
)
either_capsule_fails = (not emb_interpretation_allowed) or (not gnn_interpretation_allowed)

if either_capsule_fails or has_critical_proxy:
    overall_status = "abstain"
else:
    overall_status = "pass_exploratory"

DECISION = {
    "run_id": RUN_ID,
    "timestamp": now_utc_iso(),
    "overall_status": overall_status,
    "interpretation_allowed_embeddings": emb_interpretation_allowed,
    "interpretation_allowed_gnn": gnn_interpretation_allowed,
    "required_human_review": True,
    "verification_status": "Not verified",
    "decision_logic": {
        "schema_validation_passed": True,
        "embeddings_stability_passed": emb_interpretation_allowed,
        "gnn_stability_passed": gnn_interpretation_allowed,
        "critical_proxy_warning_present": has_critical_proxy,
        "rule_applied": (
            "abstain: either capsule failed stability or critical proxy warning present."
            if (either_capsule_fails or has_critical_proxy)
            else "pass_exploratory: both capsules stable, no critical warnings."
        )
    },
    "notes": "This decision gate is DETERMINISTIC. overall_status is derived solely from "
             "stability thresholds and guardrail triggers — no human input modifies it. "
             "required_human_review is ALWAYS true regardless of status."
}
stable_json_dump(PATHS["decision"], DECISION)

# ---------- 3. Write sensitivity report (both capsules) ----------
SENSITIVITY_REPORT = {
    "run_id": RUN_ID,
    "embeddings": SENSITIVITY_DATA.get("embeddings", []),
    "gnn": SENSITIVITY_DATA.get("gnn", []),
    "thresholds": {
        "neighbor_overlap_min": cfg["neighbor_overlap_min"],
        "score_drift_max": cfg["score_drift_max"],
        "prediction_flip_rate_max": 0.20
    },
    "verification_status": "Not verified"
}
stable_json_dump(PATHS["sensitivity_report"], SENSITIVITY_REPORT)

# ---------- 4. Write train_metrics + eval_metrics ----------
stable_json_dump(PATHS["train_metrics"], {
    "run_id": RUN_ID,
    "embeddings": TRAIN_METRICS.get("embeddings", {}),
    "gnn": TRAIN_METRICS.get("gnn", {}),
    "verification_status": "Not verified"
})
stable_json_dump(PATHS["eval_metrics"], {
    "run_id": RUN_ID,
    "gnn": EVAL_METRICS.get("gnn", {}),
    "verification_status": "Not verified"
})

# ---------- 5. Write risk_log ----------
RISK_LOG_FINAL = {
    "run_id": RUN_ID,
    "verification_status": "Not verified",
    "risk_categories_covered": [
        "relational_bias_structural_amplification",
        "contagion",
        "boundary_errors",
        "spurious_centrality_proxy_dominance",
        "accountability_diffusion"
    ],
    "events": RISK_LOG_EVENTS
}
stable_json_dump(PATHS["risk_log"], RISK_LOG_FINAL)

print("✅ Cell 8 complete — Guardrails report + Decision gate written.")
print(f"   Overall status:  {overall_status}")
print(f"   Embeddings interpretation allowed: {emb_interpretation_allowed}")
print(f"   GNN interpretation allowed:        {gnn_interpretation_allowed}")
print(f"   Required human review:             True (always)")


✅ Cell 8 complete — Guardrails report + Decision gate written.
   Overall status:  abstain
   Embeddings interpretation allowed: False
   GNN interpretation allowed:        True
   Required human review:             True (always)


##9.MODEL CARDS AND GOVERNANCE MEMOS

###9.1.0VERVIEW

**Cell 9 — Model Card + Governance Memo**

This cell writes the two most important artifacts in the bundle: the model card and the governance memo. These are not technical records of what the code did. They are human-facing documents designed to be read before anything else in the artifact bundle is examined. The conclusion of this notebook says it explicitly: read the governance memo before you read the plots. This cell is where that instruction is earned.

The model card covers both capsules under a single document. It states the intended use: teaching, demonstration, hypothesis generation for human review. It states what is out of scope: real-world decisions, production deployment, causal inference, any use without human review. The limitations section is candid. SVD captures only linear spectral structure. The GCN sees only two hops. Class imbalance inflates or deflates metrics. Stability tests cover only three perturbation types. These are not apologies. They are boundaries. A model card that omits its own limitations is not a model card. It is marketing. The edge ambiguity notes describe each edge type and what it does and does not mean. The propagation constraints summary records the hop depth, the permitted edge types, and the influence clip. Two mandatory statements close the document: structure is not causality, and connectivity is not explanation.

The governance memo is structured around seven keys that enforce a discipline of separating what is known from what is assumed. Facts\_provided records objective outcomes: the run identifier, the config hash, graph statistics, validation results, stability outcomes for both capsules, the decision status, and counts of risk and guardrail events. Assumptions lists every interpretive hypothesis the notebook has generated, each prefaced with "not verified" and each explicitly stripped of any normative claim. A hypothesis in this notebook is a statement about what the data might indicate. It is not a statement about what is true. Open\_items is a checklist of things that have not been done: data access approvals, privacy review, bias testing, domain sign-off, proxy feature review, and real-data re-validation. Analysis summarizes what was demonstrated. Draft\_output states the decision status in plain language. Questions\_to\_verify is the reviewer's checklist: eight specific questions, each targeting a governance gap that a human must close before any output moves toward production. This memo is the single artifact a reviewer should read first. Everything else in the bundle supports it.


###9.2.CODE AND IMPLEMENTATION

In [9]:

# =============================================================================
# CELL 9 — Model Card + Governance Memo (Structured, Constrained)
# =============================================================================

# ---------- 1. Model Card (covers BOTH capsules) ----------
MODEL_CARD = {
    "run_id": RUN_ID,
    "verification_status": "Not verified",

    # --- Scope ---
    "intended_use": [
        "Teaching governance of graph-based models in high-accountability settings.",
        "Demonstrating how to produce auditable artifacts for relational inference.",
        "Generating exploratory hypotheses for human review — NOT for decisions."
    ],
    "out_of_scope": [
        "Any real-world decision, ranking, or eligibility determination.",
        "Production deployment without full re-validation on real data.",
        "Causal inference from graph structure.",
        "Any use that does not include mandatory human review."
    ],

    # --- Limitations ---
    "limitations": [
        "All data is synthetic. No real-world signal is present.",
        "SVD embeddings capture only linear spectral structure; complex non-linear patterns are missed.",
        "The GCN is 2-layer (max 2 hops). Longer-range dependencies are invisible.",
        "Class imbalance (review_flag ≈ 8%) may inflate or deflate metrics. Secondary metrics should be interpreted cautiously.",
        "Sensitivity tests use only 3 perturbation types. Real-world stability may differ."
    ],

    # --- Boundary & Edge Ambiguity ---
    "boundary_summary": {
        "node_types_included": ["account", "vendor", "device"],
        "edge_types_included": ["transaction", "shared_device", "co_supply"],
        "time_window": "180 days (synthetic)",
        "exclusions": "No real data, no external sources, no PII."
    },
    "edge_ambiguity_notes": {
        "shared_device": "Ambiguous proxy — excluded from GNN adjacency. If included in embeddings, "
                         "it introduces proxy-dominance risk. See risk_log.",
        "co_supply": "Similarity edge, not causal. Vendor co-occurrence does not imply shared intent.",
        "transaction": "Synthetic frequency. Does not represent real monetary value."
    },

    # --- Propagation ---
    "propagation_constraints_summary": {
        "max_hops": cfg["gnn_max_hops"],
        "allowed_edge_types_in_gnn": list(cfg["gnn_allowed_edge_types"]),
        "influence_clip": cfg["gnn_influence_clip"],
        "normalisation": "symmetric D^{-1/2} A D^{-1/2}"
    },

    # --- Stability ---
    "stability_summary": {
        "perturbations_tested": ["edge_dropout", "edge_noise", "time_window_shift"],
        "embeddings_passed": emb_interpretation_allowed,
        "gnn_passed": gnn_interpretation_allowed
    },

    # --- Oversight ---
    "human_oversight": {
        "required": True,
        "scope": "All outputs require domain-expert review before any use.",
        "review_checklist_location": "memo/governance_memo.json → questions_to_verify"
    },

    # --- Prohibited Uses ---
    "prohibited_uses": [
        "Automated ranking or scoring for action.",
        "Eligibility or adverse-action decisions.",
        "Causal claims from structural observations.",
        "Normative labelling without independent domain validation."
    ],

    # --- Mandatory Statements ---
    "mandatory_statements": {
        "structure_is_not_causality": "Structure is not causality.",
        "connectivity_is_not_explanation": "Connectivity is not explanation."
    }
}
stable_json_dump(PATHS["model_card"], MODEL_CARD)

# ---------- 2. Governance Memo (EXACTLY 7 top-level keys) ----------
# Gather stability outcome strings
emb_stab_outcomes = []
for pr in SENSITIVITY_DATA.get("embeddings", []):
    status = "PASS" if (pr["mean_neighbor_overlap_at_k"] >= cfg["neighbor_overlap_min"] and
                        pr["mean_cosine_drift"] <= cfg["score_drift_max"]) else "FAIL"
    emb_stab_outcomes.append(f"{pr['perturbation']}: overlap={pr['mean_neighbor_overlap_at_k']}, "
                             f"drift={pr['mean_cosine_drift']} → {status}")
gnn_stab_outcomes = []
for pr in SENSITIVITY_DATA.get("gnn", []):
    status = "PASS" if (pr["mean_score_drift"] <= cfg["score_drift_max"] and
                        pr["prediction_flip_rate"] <= 0.20) else "FAIL"
    gnn_stab_outcomes.append(f"{pr['perturbation']}: drift={pr['mean_score_drift']}, "
                             f"flip={pr['prediction_flip_rate']} → {status}")

GOVERNANCE_MEMO = {
    "facts_provided": {
        "run_id": RUN_ID,
        "config_hash": CONFIG_HASH,
        "graph_stats": summarize_graph_stats(G),
        "validation_outcome": "passed — all schema rules satisfied",
        "stability_outcomes_embeddings": emb_stab_outcomes,
        "stability_outcomes_gnn": gnn_stab_outcomes,
        "decision_status": overall_status,
        "interpretation_allowed_embeddings": emb_interpretation_allowed,
        "interpretation_allowed_gnn": gnn_interpretation_allowed,
        "total_risk_events": len(RISK_LOG_EVENTS),
        "total_guardrail_events": len(GUARDRAILS_EVENTS)
    },
    "assumptions": [
        "Hypothesis (not verified): Nodes clustered together in embedding space MAY share latent behavioural similarity. "
        "This does NOT confirm shared intent, collusion, or any normative property.",
        "Hypothesis (not verified): The hub node's high centrality MAY reflect structural importance within the synthetic graph, "
        "but this is an artefact of the generation process and carries no real-world meaning.",
        "Hypothesis (not verified): The ambiguous connector's many shared_device edges MAY indicate a co-location pattern, "
        "but shared device access does NOT imply shared ownership or coordinated behaviour.",
        "Hypothesis (not verified): GNN predictions on the synthetic review_flag label MAY reflect learnable neighbourhood patterns, "
        "but these patterns are artifacts of the synthetic data generator.",
        "Assumption: The synthetic time window (180 days) is arbitrary. A different window would produce different graph structure."
    ],
    "open_items": [
        "Data access approvals: No real data has been used. Before any real-world application, formal data-access and privacy review is required.",
        "Privacy review: PII handling, consent, and data-minimisation policies must be assessed.",
        "Bias testing: Systematic fairness testing across protected characteristics is required before any deployment.",
        "Domain sign-off: A subject-matter expert must validate edge semantics, boundary definitions, and feature provenance.",
        "Proxy feature review: synth_proxy_device_count is flagged as a proxy. Its inclusion requires explicit domain justification.",
        "Real-data re-validation: All schemas, thresholds, and stability tests must be re-run on real data before production use."
    ],
    "analysis": (
        "This notebook demonstrates governance of two graph-based models on synthetic data. "
        "Capsule A (Graph Embeddings) converts adjacency structure into dense node representations via SVD, "
        "then tests stability under edge dropout, noise, and time-window shift. "
        "Capsule B (GNN) performs 2-hop message passing on a filtered adjacency (excluding ambiguous shared_device edges) "
        "and trains a node classifier on a synthetic label. Both capsules produce sensitivity reports that drive "
        "a deterministic abstention gate. The decision status reflects whether stability thresholds were met. "
        "All outputs are hypotheses requiring human verification. No causal or normative claims are made."
    ),
    "draft_output": (
        "Draft output (Not verified): The notebook has completed end-to-end execution. "
        f"Decision status = '{overall_status}'. "
        f"Embeddings interpretation allowed = {emb_interpretation_allowed}. "
        f"GNN interpretation allowed = {gnn_interpretation_allowed}. "
        "All artifact files have been written to the artifact bundle. "
        "A human reviewer should examine questions_to_verify before any further use."
    ),
    "verification_status": "Not verified",
    "questions_to_verify": [
        "Boundary validity: Are the included node and edge types appropriate for the intended use case? "
        "Should any types be added or removed?",
        "Edge semantics: Do the edge-type definitions (transaction, shared_device, co_supply) accurately reflect "
        "the intended real-world relationships? Are the ambiguity notes complete?",
        "Missingness: Is the 'missing = uninformative' assumption valid, or could missing edges carry signal?",
        "Propagation radius: Is 2-hop message passing appropriate, or should max_hops be adjusted based on domain knowledge?",
        "Sensitivity results: Are the observed stability outcomes acceptable for the intended use? "
        "Should thresholds be tightened or relaxed?",
        "Proxy risks: Is synth_proxy_device_count (or any real-world analogue) acceptable as a node feature? "
        "What domain evidence supports or refutes its inclusion?",
        "Centrality sanity: Has the hub node been accounted for in interpretation? "
        "Are there other high-degree nodes whose centrality is spurious?",
        "No decision authority: Confirm that no downstream system will use these outputs to make or influence "
        "automated decisions without explicit human review at each step."
    ]
}
stable_json_dump(PATHS["governance_memo"], GOVERNANCE_MEMO)

print("✅ Cell 9 complete — Model card and governance memo written.")
print(f"   Memo top-level keys: {list(GOVERNANCE_MEMO.keys())}")


✅ Cell 9 complete — Model card and governance memo written.
   Memo top-level keys: ['facts_provided', 'assumptions', 'open_items', 'analysis', 'draft_output', 'verification_status', 'questions_to_verify']


##10.AUDIT BUNDLE

###10.1.OVERVIEW

**Cell 10 — Sample Outputs + Refusal Example + Zip Bundle + Summary**

This cell is the end of execution, and it closes the notebook with three acts. The first is producing sample outputs that demonstrate what the notebook is willing to say and what it refuses to say. The second is packaging every artifact into a zip bundle. The third is printing a run summary that functions as a governance dashboard.

The sample outputs include embedding neighbor hypotheses for four anchor nodes. Each hypothesis states which nodes appeared closest in embedding space, explicitly labels the observation as unverified, and explicitly states that proximity does not imply shared behavior, intent, or risk. A community structural summary describes the connected components of the graph: how many exist, how large the largest is. It labels this observation as structural description only and explicitly states that community membership does not imply coordinated activity or any normative property.

The refusal example is the most pedagogically important element in this cell. A prohibited question is posed: "Who should we investigate?" The notebook refuses. The refusal text states that the notebook does not identify entities for investigation, ranking, or adverse action. It states that graph structure is observational only. It states that interpreting structure as actionable guidance violates the governance boundary. It closes with the two mandatory sentences: structure is not causality, connectivity is not explanation. The refusal is recorded as an enforcement event in the guardrails report. It is not an afterthought. It is a demonstration that the governance boundary is real and that it is enforced, not merely stated.

The zip bundle packages all eighteen required artifact files into a single compressed archive. A verification loop then checks that every required file exists on disk, printing a checkmark or a failure marker beside each one. If any artifact is missing, the summary flags it immediately. The run summary prints the run identifier, the config hash, the overall decision status, the interpretation flags for both capsules, the human review requirement, and the full artifact checklist. It closes with a reminder that all outputs are unverified and that human review is mandatory. The notebook does not end with a result. It ends with an obligation. The artifacts are complete. The decision has been made by the gate. What happens next — whether these hypotheses are examined, whether a domain expert reviews the memo, whether any output reaches a production system — is entirely in human hands. That handoff is the point of the entire notebook.

###10.2.CODE AND IMPLEMENTATION

In [10]:

# =============================================================================
# CELL 10 — Sample Outputs + Refusal Example + Zip Bundle + Print Summary
# =============================================================================
import zipfile

# ---------- 1. Build sample outputs ----------
# (a) Embedding neighbor hypotheses (a few anchors)
embedding_neighbor_hypotheses = []
for ai in anchor_indices[:4]:   # show 4 examples
    nid = all_node_ids_ordered[ai]
    ntype = G.nodes[nid].get("node_type", "unknown")
    nbr_ids = [all_node_ids_ordered[ni] for ni in sorted(base_neighbors[ai])]
    nbr_types = [G.nodes[n].get("node_type", "unknown") for n in nbr_ids]
    embedding_neighbor_hypotheses.append({
        "anchor_node": nid,
        "anchor_type": ntype,
        "hypothesis": f"Nodes {nbr_ids} (types: {nbr_types}) appear closest in embedding space to {nid}. "
                      "This MAY indicate shared structural neighbourhood but is NOT verified and does NOT "
                      "imply shared behaviour, intent, or risk. This is a hypothesis only.",
        "verification_status": "Not verified"
    })

# (b) Community / structural summary (structural description only)
# Compute connected components on undirected version for illustration
G_undir = G.to_undirected()
components = list(nx.connected_components(G_undir))
components_sorted = sorted(components, key=len, reverse=True)
community_summary = {
    "n_components": len(components_sorted),
    "largest_component_size": len(components_sorted[0]) if components_sorted else 0,
    "smallest_component_size": len(components_sorted[-1]) if components_sorted else 0,
    "structural_description": (
        f"The graph contains {len(components_sorted)} connected component(s). "
        f"The largest spans {len(components_sorted[0])} nodes. "
        "This is a structural observation only. Community membership does NOT imply shared behaviour, "
        "coordinated activity, or any normative property. This is a hypothesis. Not verified."
    ),
    "verification_status": "Not verified"
}

# (c) Refusal example — user asks a prohibited question
PROHIBITED_QUESTION = "Who should we investigate?"
REFUSAL_TEXT = (
    "REFUSED. This notebook does not identify entities for investigation, ranking, or any form of "
    "adverse action. Graph structure (centrality, community, similarity) is observational only. "
    "Interpreting structure as actionable guidance violates the governance boundary of this notebook. "
    "See guardrails_report.json for allowed and prohibited uses. "
    "All outputs are labeled 'Not verified' and require human domain review before any use. "
    "Structure is not causality. Connectivity is not explanation."
)

# Record the refusal as an enforcement event
refusal_enforcement_event = {
    "event_type": "boundary_refusal",
    "prohibited_query": PROHIBITED_QUESTION,
    "refusal_text": REFUSAL_TEXT,
    "timestamp": now_utc_iso(),
    "verification_status": "Not verified"
}
GUARDRAILS_EVENTS.append(refusal_enforcement_event)

# Update guardrails report with the refusal event
guardrails_data = json.load(open(PATHS["guardrails_report"], "r"))
guardrails_data["enforcement_events"].append(refusal_enforcement_event)
stable_json_dump(PATHS["guardrails_report"], guardrails_data)

# ---------- 2. Write sample_outputs.json ----------
# Include embedding matrix hash for traceability
emb_hash = sha256_dict({"embedding_matrix_shape": list(emb_base.shape),
                         "embedding_matrix_checksum": float(emb_base.sum())})

SAMPLE_OUTPUTS = {
    "run_id": RUN_ID,
    "verification_status": "Not verified",
    "embedding_neighbor_hypotheses": embedding_neighbor_hypotheses,
    "community_structural_summary": community_summary,
    "embedding_matrix_trace": {
        "shape": list(emb_base.shape),
        "hash": emb_hash,
        "note": "Full embedding matrix not serialised here for size. Hash provides integrity check."
    },
    "refusal_example": {
        "user_query": PROHIBITED_QUESTION,
        "response": REFUSAL_TEXT,
        "enforcement_event_recorded": True,
        "verification_status": "Not verified"
    },
    "gnn_test_predictions_summary": {
        "n_test_accounts": int(test_mask.sum().item()),
        "predicted_0": int((test_preds == 0).sum()),
        "predicted_1": int((test_preds == 1).sum()),
        "note": "Predictions are on synthetic labels only. Not verified. Must not drive decisions.",
        "verification_status": "Not verified"
    }
}
stable_json_dump(PATHS["sample_outputs"], SAMPLE_OUTPUTS)

# ---------- 3. Create artifacts_bundle.zip ----------
zip_path = PATHS["bundle_zip"]
with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zf:
    for root, dirs, files in os.walk(ARTIFACT_ROOT):
        for fname in files:
            if fname == "artifacts_bundle.zip":
                continue   # don't include the zip inside itself
            fpath = os.path.join(root, fname)
            arcname = os.path.relpath(fpath, os.path.dirname(ARTIFACT_ROOT))
            zf.write(fpath, arcname)

# ---------- 4. Verify all required artifact files exist ----------
REQUIRED_ARTIFACTS = [
    "run_manifest.json",
    os.path.join("schemas", "node_schema.json"),
    os.path.join("schemas", "edge_schema.json"),
    os.path.join("schemas", "graph_construction_manifest.json"),
    os.path.join("validation", "data_validation_log.json"),
    os.path.join("split", "split_manifest.json"),
    os.path.join("metrics", "baseline_metrics.json"),
    os.path.join("metrics", "train_metrics.json"),
    os.path.join("metrics", "eval_metrics.json"),
    os.path.join("reports", "propagation_constraints.json"),
    os.path.join("reports", "sensitivity_report.json"),
    os.path.join("reports", "guardrails_report.json"),
    os.path.join("model", "model_card.json"),
    os.path.join("decision", "decision.json"),
    os.path.join("risk", "risk_log.json"),
    os.path.join("memo", "governance_memo.json"),
    os.path.join("outputs", "sample_outputs.json"),
    "artifacts_bundle.zip"
]

print("\n" + "=" * 70)
print("  CHAPTER 4 — RUN SUMMARY")
print("=" * 70)
print(f"  RUN_ID              : {RUN_ID}")
print(f"  CONFIG_HASH         : {CONFIG_HASH[:24]}…")
print(f"  OVERALL STATUS      : {overall_status}")
print(f"  Embeddings interp.  : {'ALLOWED' if emb_interpretation_allowed else 'BLOCKED'}")
print(f"  GNN interp.         : {'ALLOWED' if gnn_interpretation_allowed else 'BLOCKED'}")
print(f"  Human review req.   : True (always)")
print(f"  Verification status : Not verified")
print("-" * 70)
print("  ARTIFACT CHECK:")
all_present = True
for req in REQUIRED_ARTIFACTS:
    full_path = os.path.join(ARTIFACT_ROOT, req)
    exists = os.path.isfile(full_path)
    status_icon = "✅" if exists else "❌"
    print(f"    {status_icon}  {req}")
    if not exists:
        all_present = False
print("-" * 70)
print(f"  All required artifacts present: {'YES ✅' if all_present else 'NO ❌ — CHECK ABOVE'}")
print(f"  Bundle zip           : {zip_path}")
print(f"  Bundle size          : {os.path.getsize(zip_path):,} bytes")
print("=" * 70)
print("\n⚠️  All outputs are 'Not verified'. Human review is mandatory.")
print("    Structure is not causality. Connectivity is not explanation.")
print("=" * 70)



  CHAPTER 4 — RUN SUMMARY
  RUN_ID              : run_20260201T142447_d6e413a4
  CONFIG_HASH         : 6e7c6b6060052e638098e3db…
  OVERALL STATUS      : abstain
  Embeddings interp.  : BLOCKED
  GNN interp.         : ALLOWED
  Human review req.   : True (always)
  Verification status : Not verified
----------------------------------------------------------------------
  ARTIFACT CHECK:
    ✅  run_manifest.json
    ✅  schemas/node_schema.json
    ✅  schemas/edge_schema.json
    ✅  schemas/graph_construction_manifest.json
    ✅  validation/data_validation_log.json
    ✅  split/split_manifest.json
    ✅  metrics/baseline_metrics.json
    ✅  metrics/train_metrics.json
    ✅  metrics/eval_metrics.json
    ✅  reports/propagation_constraints.json
    ✅  reports/sensitivity_report.json
    ✅  reports/guardrails_report.json
    ✅  model/model_card.json
    ✅  decision/decision.json
    ✅  risk/risk_log.json
    ✅  memo/governance_memo.json
    ✅  outputs/sample_outputs.json
    ✅  artifacts_bu

##11.CONCLUSION

**Conclusion: What the Graph Taught Us About Governance**

This notebook is finished. The artifacts are written, the bundle is zipped, the decision gate has spoken. But the notebook finishing is not the same as the work finishing. Everything produced here — every embedding, every prediction, every hypothesis in the sample outputs — is raw material. It becomes useful only when a human picks it up, reads the governance memo, and decides what to do next. This conclusion walks through what we actually built, what it revealed, how it differs from what came before, and where the responsibility now shifts to you.

**How We Pursued the Three Objectives**

The first objective was to make the conversion from adjacency to signal visible and auditable. We did that by choosing the simplest possible embedding method — truncated SVD on the adjacency matrix — and the simplest possible GNN — a two-layer graph convolutional network written by hand in PyTorch, no library abstractions. These choices were not limitations. They were governance decisions. When a method is simple enough to trace by hand, you can reason about exactly what it is doing to the data. You can point to the adjacency matrix, point to the SVD, and say: this is where the structure became a number. That traceability is what the artifact bundle captures. The train metrics record what went in. The embedding hash in the sample outputs confirms what came out. The gap between those two artifacts is where a human reviewer can ask: does this transformation make sense for the domain?

The second objective was to surface relational risk in a way that could not be ignored. We did not describe propagation risk, boundary risk, and proxy risk in the abstract. We built them into the synthetic graph. The hub node exists so that you can see, in the PCA plot, what a high-degree node looks like in embedding space — and then ask whether its position there is meaningful or an artifact of degree. The connector node exists so that you can see what happens when ambiguous edges (shared devices) enter the embedding input — and then check whether the propagation constraints artifact correctly excluded them from the GNN. The time-window shift test exists so that you can see what happens when the boundary moves — and then decide whether the stability results are acceptable for whatever real-world use case you have in mind. Each risk was not just flagged. It was demonstrated, measured, and recorded.

The third objective was to show that governance artifacts constrain interpretation rather than merely documenting it after the fact. The decision gate in Cell 8 is deterministic. It reads the sensitivity report, compares values to thresholds, and sets the overall status. No human touches that logic. If stability fails, interpretation is blocked. The system does not suggest caution. It does not add a disclaimer. It stops. The guardrails report records every enforcement event, including the refusal example in Cell 10, where the notebook was asked who should be investigated and declined to answer. That refusal is not an edge case. It is the boundary of the system made visible.

**What This Chapter Added That Previous Chapters Did Not**

Chapters 1 through 3 governed data that lived in tables. Each row was an independent observation. Bias in one row did not infect another. A bad feature in one sample did not propagate to its neighbors. Chapter 4 broke that assumption. In a graph, every node is a function of its neighborhood. That single change — from independence to interdependence — introduced three governance problems that did not exist before.

The first is propagation. In Chapter 2, a proxy feature could mislead the model about a single prediction. In Chapter 4, a proxy edge can mislead the model about every node within two hops of where the proxy enters the graph. The GNN does not see individual edges. It sees aggregated neighborhood signals. A single ambiguous edge, repeated across a cluster, becomes a structural bias that the model cannot distinguish from a real pattern. The propagation constraints artifact is the answer: it records which edge types are permitted in message passing, how many hops are allowed, and why. Without that artifact, there is no way to audit whether the GNN's neighborhood was clean.

The second is boundary. In Chapter 1, the data was a fixed table. The boundary was defined by which columns were included. In Chapter 4, the boundary is defined by which edges exist, which timestamps fall within the window, and which node types are in scope. Every one of those choices changes the graph the model sees. The graph construction manifest records those choices. The time-window shift test measures what happens when one of them changes. If the model's output shifts significantly when the boundary moves, the output is boundary-dependent — it is a function of when you looked, not just what you looked at. That is a different kind of instability than anything in Chapters 1 through 3, and it required a different kind of test.

The third is proxy dominance through edge ambiguity. In Chapter 2, proxy features were detectable because they were columns in a table. You could inspect them, flag them, remove them. In Chapter 4, proxy edges are structural. A shared\_device edge is not a column you can drop. It is a connection that participates in the topology of the graph. Removing it changes the graph. Keeping it introduces ambiguity into every embedding and every aggregation that touches it. The notebook handled this by excluding shared\_device from the GNN adjacency entirely — a governance rule, not a modeling choice — while leaving it in the embedding input to demonstrate the risk. The edge schema documents the ambiguity. The risk log records the warning. The human reviewer must decide whether that exclusion is correct for their domain.

**What the Results Actually Mean**

The embedding PCA plot showed three clusters corresponding to the three node types. That is expected. The adjacency structure separates accounts from vendors from devices because they connect differently. The plot did not reveal hidden communities or suspicious subgroups. It revealed what the synthetic graph was designed to reveal: that structure becomes visible when projected into embedding space, and that visibility does not equal meaning.

The GNN training loss decreased. The confusion matrix showed heavy prediction of the majority class. The test accuracy and F1 scores were modest. None of that is surprising. The review\_flag label was assigned at random to eight percent of accounts. There is no real signal in the neighborhood structure for the GNN to find. The metrics are not evidence of model quality. They are evidence that the governance workflow ran correctly: the model trained, it produced outputs, and those outputs were measured against a held-out set.

The stability tests produced the most consequential results. Whether the decision gate landed on "abstain" or "pass\_exploratory" depends entirely on whether the sensitivity metrics breached their thresholds. If the status is "abstain," interpretation is blocked — not because something failed catastrophically, but because the model's outputs shifted enough under perturbation that trusting them without human review would be irresponsible. If the status is "pass\_exploratory," interpretation is permitted — but only as hypothesis generation, and only after a human has reviewed the questions listed in the governance memo.

**Where the Responsibility Now Shifts to You**

The notebook executed every check it was designed to execute. It validated the schema. It measured stability. It enforced guardrails. It wrote eighteen artifact files. It refused to answer a prohibited question. All of that is automated. All of that is deterministic. None of it requires human judgment.

What requires human judgment is everything that comes after. The governance memo lists the open items explicitly. Data access approvals have not been obtained — this notebook used synthetic data. Privacy review has not been performed. Bias testing across protected characteristics has not been conducted. Domain sign-off on edge semantics has not happened. The proxy feature (synth\_proxy\_device\_count) has been flagged but not independently evaluated. The questions\_to\_verify section of the memo is a checklist. Every item on it must be resolved by a person before any output from this notebook moves closer to production.

The model card states two sentences that govern everything this notebook produced: **structure is not causality**, and **connectivity is not explanation**. No graph observation — no cluster, no centrality score, no embedding proximity — can be used as evidence that something is true about the real world. It can be used as a hypothesis. It can be used to decide where a human expert should look next. It cannot be used to decide what to do. The notebook does not cross that boundary. It hands you the artifacts, points to the memo, and waits.

**Capability ↑ ⇒ Risk ↑ ⇒ Controls must ↑.** Chapter 4 added relational capability. It added relational risk. It added relational controls. The pattern holds. It will hold in Chapter 5 as well. The models will get more capable. The risks will get more subtle. The controls will get more demanding. And at every stage, the notebook will do what it was built to do: produce the evidence, enforce the boundaries, and hand the decisions to you.